------
**You cannot apply any changes to this file, so please make sure to save it on your Google Colab drive or download it as a .ipynb file.**

------


# Practical 2: Representing Sentences with Neural Models

In this second practical, we will train neural network models to obtain sentence representations. We can then use these sentence representations for a downstream task such as sentiment classification. 

In this notebook, we will help you to develop models for your experiments. But this time, next to completing the notebook, **you are expected to write a four-page scientific report with your findings**. Please still submit the notebook together with your scientific report so that we can reproduce your experiments. (Note: if you find it useful, you can split this notebook into multiple notebooks. If you do so, keep it mind that it should be possible for your TAs to reproduce the entire content of the notebooks without having to ask for clarifications or to copy and paste functions from one sub-notebook to another.)

**Important!** The main purpose of this lab is for you to learn how to answer research questions by experimenting and then writing a scientific report.
So you will be *judged by the quality of your report* but will lose points if your experiments are not reproducible.
You can find the requirements for the report at the end of this notebook.


### Data set
We will use the [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/) (SST), which provides sentences, their binary tree structure, and fine-grained sentiment scores.
This dataset is different from the one we used in the first practical. 
In Practical 1, a review consisted of several sentences, and we had one sentiment score for the whole review. Now, a review consists of a single sentence, and we have a sentiment score for each node in the binary tree that makes up the sentence, including the root node (i.e., we still have an overall sentiment score for the entire review). We will look at an example below.

In the first part of this practical we will only make use of the sentence tokens whereas in the second part we will also exploit the tree structure that is provided by the SST.

We will cover the following approaches:

- Bag-of-words (BOW)
- Continuous bag-of-words (CBOW)
- Deep continuous bag-of-words (Deep CBOW)
- LSTM
- Tree-LSTM


## Preparing the data

Let's first download the data set and take a look.

In [68]:
import re
import random
import time
import math
import numpy as np
import nltk
import matplotlib.pyplot as plt
plt.style.use('default')

In [2]:
!wget http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
!unzip trainDevTestTrees_PTB.zip

--2020-11-20 09:26:38--  http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip [following]
--2020-11-20 09:26:38--  https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789539 (771K) [application/zip]
Saving to: 'trainDevTestTrees_PTB.zip'

     0K .......... .......... .......... .......... ..........  6%  134K 5s
    50K .......... .......... .......... .......... .......... 12%  301K 4s
   100K .......... .......... .......... .......... .......... 19%  389K 3s
   150K .......... .......... .......... .......... .......... 25%  305K 2s
   200K .......... .......... .......

In [69]:
# this function reads in a textfile and fixes an issue with "\\"
def filereader(path): 
  with open(path, mode="r", encoding="utf-8") as f:
    for line in f:
      yield line.strip().replace("\\","")

Let's look at a data point. It is a **flattened binary tree**, with sentiment scores at every node, and words as the leaves (or *terminal nodes*).

In [70]:
s = next(filereader("trees/dev.txt"))
print(s)

(3 (2 It) (4 (4 (2 's) (4 (3 (2 a) (4 (3 lovely) (2 film))) (3 (2 with) (4 (3 (3 lovely) (2 performances)) (2 (2 by) (2 (2 (2 Buy) (2 and)) (2 Accorsi))))))) (2 .)))


In [71]:
# We can use NLTK to better visualise the tree structure of the sentence
from nltk import Tree
from nltk.treeprettyprinter import TreePrettyPrinter
tree = Tree.fromstring(s)
print(TreePrettyPrinter(tree))

              3                                                                     
  ____________|____________________                                                  
 |                                 4                                                
 |        _________________________|______________________________________________   
 |       4                                                                        | 
 |    ___|______________                                                          |  
 |   |                  4                                                         | 
 |   |         _________|__________                                               |  
 |   |        |                    3                                              | 
 |   |        |               _____|______________________                        |  
 |   |        |              |                            4                       | 
 |   |        |              |            ________________|_

The sentiment scores range from 0 (very negative) to 5 (very positive). Again, as you can see, every node in the tree is labeled with a sentiment score. For now, we will only use the score at the **root node**, i.e., the sentiment score for the complete sentence.

In [72]:
# Let's first make a function that extracts the tokens (the leaves).

def tokens_from_treestring(s):
  """extract the tokens from a sentiment tree"""
  return re.sub(r"\([0-9] |\)", "", s).split()
 
# let's try it on our example tree
tokens = tokens_from_treestring(s)
print(tokens)
print(len(tokens))

['It', "'s", 'a', 'lovely', 'film', 'with', 'lovely', 'performances', 'by', 'Buy', 'and', 'Accorsi', '.']
13


> *Warning: you could also parse a treestring using NLTK and ask it to return the leaves, but there seems to be an issue with NLTK not always correctly parsing the input, so do not rely on it.*

In [73]:
# We will also need the following function, but you can ignore this for now.
# It is explained later on.

SHIFT = 0
REDUCE = 1


def transitions_from_treestring(s):
  s = re.sub("\([0-5] ([^)]+)\)", "0", s)
  s = re.sub("\)", " )", s)
  s = re.sub("\([0-4] ", "", s)
  s = re.sub("\([0-4] ", "", s)
  s = re.sub("\)", "1", s)
  return list(map(int, s.split()))

In [74]:
# Now let's first see how large our data sets are.
for path in ("trees/train.txt", "trees/dev.txt", "trees/test.txt"):
  print("{:16s} {:4d}".format(path, sum(1 for _ in filereader(path))))

trees/train.txt  8544
trees/dev.txt    1101
trees/test.txt   2210


You can see that the number of sentences is not very large. That's probably because the data set required so much manual annotation. However, it is large enough to train a neural network on.

It will be useful to store each data example in an `Example` object,
containing everything that we may need for each data point.
It will contain the tokens, the tree, the top-level sentiment label, and 
the transitions (explained later).

In [75]:
from collections import namedtuple
from nltk import Tree

# A simple way to define a class is using namedtuple.
Example = namedtuple("Example", ["tokens", "tree", "label", "transitions"])

   
def examplereader(path, lower=False):
  """Returns all examples in a file one by one."""
  for line in filereader(path):
    line = line.lower() if lower else line
    tokens = tokens_from_treestring(line)
    tree = Tree.fromstring(line)  # use NLTK's Tree
    label = int(line[1])
    trans = transitions_from_treestring(line)
    yield Example(tokens=tokens, tree=tree, label=label, transitions=trans)
  

# Let's load the data into memory.
LOWER = False  # we will keep the original casing
train_data = list(examplereader("trees/train.txt", lower=LOWER))
dev_data = list(examplereader("trees/dev.txt", lower=LOWER))
test_data = list(examplereader("trees/test.txt", lower=LOWER))

print("train", len(train_data))
print("dev", len(dev_data))
print("test", len(test_data))

train 8544
dev 1101
test 2210


In [178]:
from nltk import pos_tag

sentence_pos_tags = [pos_tag(sentence.tokens) for sentence in train_data]




In [206]:
import random

def pos_remover(tags, prob):
    
    new_data = []
    
    for pos_sentence, train_sentence in zip(sentence_pos_tags, train_data):
#         print(pos_sentence, train_sentence.tokens)
        
        sentence = []
        for pos_word , train_word in zip(pos_sentence, train_sentence.tokens):

            for tag in tags:
                if str(pos_word[1]).startswith(tag) and random.uniform(0, 1) < prob:
                    continue
                else:
                    sentence.append(pos_word[0])

        new_data.append(sentence)
        
    return new_data

                

75_remove_noun_etc = pos_remover(["NN", "RB", "JJ"], 0.75)
50_remove_noun_etc = pos_remover(["NN", "RB", "JJ"], 0.50)

# adverb = rb
# adjective = JJ



SyntaxError: invalid token (<ipython-input-206-f08349f70042>, line 25)

In [194]:
new_bende

[['Little', 'more', 'than', 'a', 'well-mounted', '.'],
 ['What',
  'makes',
  'Esther',
  'Kahn',
  'so',
  'demanding',
  'is',
  'that',
  'it',
  'progresses',
  'in',
  'such',
  'a',
  'low-key',
  'that',
  'it',
  'risks',
  '.'],
 ['It',
  'may',
  'not',
  'be',
  'a',
  'great',
  'of',
  ',',
  'but',
  'its',
  'comes',
  'from',
  'its',
  "'s",
  '-',
  'of',
  'how',
  'well-meaning',
  'masked',
  'a',
  'social',
  ',',
  'at',
  'least',
  'as',
  'represented',
  'by',
  'this',
  '.'],
 ['It',
  "'s",
  'got',
  'some',
  'pretentious',
  'eye-rolling',
  'moments',
  'and',
  'it',
  'did',
  "n't",
  'entirely',
  'grab',
  'me',
  ',',
  'but',
  'there',
  "'s",
  'here',
  'to',
  'like',
  '.'],
 ['Truly', 'terrible', '.'],
 ['A',
  'cheap',
  'put',
  'together',
  'by',
  'some',
  'cynical',
  'creeps',
  'at',
  'Revolution',
  'Studios',
  'and',
  'Imagine',
  'Entertainment',
  'to',
  'make',
  'the',
  'suckers',
  'out',
  'there',
  'surrender',
  '

In [135]:
lengtes = []
for data in train_data:
    lengtes.append(len(data.tokens))
print(np.average(lengtes))
print(np.std(lengtes))
kort = []
lang = []
for lengte in lengtes:
    if lengte <= np.average(lengtes) - np.std(lengtes):
        kort.append(lengte)
    if lengte >= np.average(lengtes) + np.std(lengtes):
        lang.append(lengte)
print(np.average(kort), len(kort))
print(np.average(lang),len(lang))


19.143960674157302
9.305588412283663
6.406791907514451 1384
34.22309337134711 1403


In [139]:
short_train = []
long_train = []
for data in train_data:
    if len(data.tokens) <= np.average(lengtes) - np.std(lengtes):
        short_train.append(data)
    if len(data.tokens) >= np.average(lengtes) + np.std(lengtes):
        long_train.append(data)


In [140]:
print(len(short_train),len(long_train))

1384 1403


Let's check out an `Example` object.

In [ ]:
example = dev_data[0]
print("First example:", example)
print("First example tokens:", example.tokens)
print("First example label:",  example.label)

#### Vocabulary 
A first step in most NLP tasks is collecting all the word types that appear in the data into a vocabulary, and counting the frequency of their occurrences. On the one hand, this will give us an overview of the word distribution of the data set (what are the most frequent words, how many rare words are there, ...). On the other hand, we will also use the vocabulary to map each word to a unique numeric ID, which is a more handy index than a string.

In [ ]:
# Here we first define a class that can map a word to an ID (w2i)
# and back (i2w).

from collections import Counter, OrderedDict, defaultdict


class OrderedCounter(Counter, OrderedDict):
  """Counter that remembers the order elements are first seen"""
  def __repr__(self):
    return '%s(%r)' % (self.__class__.__name__,
                      OrderedDict(self))
  def __reduce__(self):
    return self.__class__, (OrderedDict(self),)


class Vocabulary:
  """A vocabulary, assigns IDs to tokens"""
  
  def __init__(self):
    self.freqs = OrderedCounter()
    self.w2i = {}
    self.i2w = []

  def count_token(self, t):
    self.freqs[t] += 1
    
  def add_token(self, t):
    self.w2i[t] = len(self.w2i)
    self.i2w.append(t)    
    
  def build(self, min_freq=0):
    '''
    min_freq: minimum number of occurrences for a word to be included  
              in the vocabulary
    '''
    self.add_token("<unk>")  # reserve 0 for <unk> (unknown words)
    self.add_token("<pad>")  # reserve 1 for <pad> (discussed later)   
    
    tok_freq = list(self.freqs.items())
    tok_freq.sort(key=lambda x: x[1], reverse=True)
    for tok, freq in tok_freq:
      if freq >= min_freq:
        self.add_token(tok)

The vocabulary has by default an `<unk>` token and a `<pad>` token. The `<unk>` token is reserved for all words which do not appear in the training data (and for which, therefore, we cannot learn word representations). The function of the `<pad>` token will be explained later.


Let's build the vocabulary!


In [ ]:
# This process should be deterministic and should have the same result 
# if run multiple times on the same data set.

v = Vocabulary()
for data_set in (train_data,):
  for ex in data_set:
    for token in ex.tokens:
      v.count_token(token)

v.build()
print("Vocabulary size:", len(v.w2i))

Let's have a closer look at the properties of our vocabulary. Having a good idea of what it is like can facilitate data analysis and debugging later on.

In [ ]:
# What is the ID for "century?"
print(v.w2i['century'])


In [ ]:
# What are the first 10 words in the vocabulary (based on their IDs)?
print(v.i2w[:10])

In [ ]:
# What are the 10 most common words?
sortedfreqs = sorted(v.freqs.items(), key=lambda x: x[1], reverse=True)

i = 0
for key,value in sortedfreqs:
    print(key,value)
    i += 1
    if i == 10:
        break

In [ ]:
# And how many words are there with frequency 1?
# (A fancy name for these is hapax legomena.)
i = 0
for key,value in sortedfreqs:
    if value == 1:
        i += 1
print(i)

In [ ]:
# Finally 20 random words from the vocabulary.
# This is a simple way to get a feeling for the data. 
# You could use the `choice` function from the already imported `random` package
i = 0
while i < 20:
    print(random.choice(v.i2w))
    i += 1


#### Sentiment label vocabulary

In [ ]:
# Now let's map the sentiment labels 0-4 to a more readable form
i2t = ["very negative", "negative", "neutral", "positive", "very positive"]
print(i2t)
print(i2t[4])

In [ ]:
# And let's also create the opposite mapping.
# We won't use a Vocabulary for this (although we could), since the labels
# are already numeric.
t2i = OrderedDict({p : i for p, i in zip(i2t, range(len(i2t)))})
print(t2i)
print(t2i['very positive'])

## PyTorch

In Colab notebooks, the last available version of PyTorch is already installed.The current stable version is 1.7.

*For installing PyTorch in your own computer, follow the instructions on [pytorch.org](pytorch.org) instead. This is for Google Colab only.*

In [ ]:
import torch
print("Using torch", torch.__version__) # should say 1.7.0+cu101

In [ ]:
# Let's also import torch.nn, a PyTorch package that  
# makes building neural networks more convenient.
from torch import nn

In [ ]:
# PyTorch can run on CPU or on Nvidia GPU (video card) using CUDA
# This cell selects the GPU if one is available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# Seed manually to make runs reproducible
# You need to set this again if you do multiple runs of the same model
torch.manual_seed(0)

# When running on the CuDNN backend two further options must be set for reproducibility
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# BOW

Our first model is a rather simple neural **bag-of-words (BOW) model**.
Unlike the bag-of-words model that you used in the previous lab, where we would look at the presence / frequency of words in a text, here we associate each word with a multi-dimensional vector which expresses what sentiment is conveyed by the word. In particular, our BOW vectors will be of size 5, exactly our number of sentiment classes. 

To classify a sentence, we **sum** the vectors of the words in the sentence and a bias vector. Because we sum the vectors, we lose word order: that's why we call this a neural bag-of-words model.

```
this   [0.0, 0.1, 0.1, 0.1, 0.0]
movie  [0.0, 0.1, 0.1, 0.2, 0.1]
is     [0.0, 0.1, 0.0, 0.0, 0.0]
stupid [0.9, 0.5, 0.1, 0.0, 0.0]

bias   [0.0, 0.0, 0.0, 0.0, 0.0]
--------------------------------
sum    [0.9, 0.8, 0.3, 0.3, 0.1]

argmax: 0 (very negative)
```

The **argmax** of this sum is our predicted label.

We initialize all vectors *randomly* and train them using cross-entropy loss.

#### Model class

In [ ]:
class BOW(nn.Module):
  """A simple bag-of-words model"""

  def __init__(self, vocab_size, embedding_dim, vocab):
    super(BOW, self).__init__()
    self.vocab = vocab
    
    # this is a trainable look-up table with word embeddings
    self.embed = nn.Embedding(vocab_size, embedding_dim)
    
    # this is a trainable bias term
    self.bias = nn.Parameter(torch.zeros(embedding_dim), requires_grad=True)        

  def forward(self, inputs):
    # this is the forward pass of the neural network
    # it applies a function to the input and returns the output

    # this looks up the embeddings for each word ID in inputs
    # the result is a sequence of word embeddings
    embeds = self.embed(inputs)

    
    # the output is the sum across the time dimension (1)
    # with the bias term added
    logits = embeds.sum(1) + self.bias

    return logits

In [ ]:
# Let's create a model.
vocab_size = len(v.w2i)
n_classes = len(t2i)
bow_model = BOW(vocab_size, n_classes, v)
print(bow_model)



> **Hey, wait, where is the bias vector?**
> PyTorch does not print Parameters, only Modules!

> We can print it ourselves though, to check that it is there.

In [77]:

# Here we print each parameter name, shape, and if it is trainable.
def print_parameters(model):
  total = 0
  for name, p in model.named_parameters():
    total += np.prod(p.shape)
    print("{:24s} {:12s} requires_grad={}".format(name, str(list(p.shape)), p.requires_grad))
  print("\nTotal number of parameters: {}\n".format(total))
    

#print_parameters(bow_model)

#### Preparing an example for input

To feed sentences to our PyTorch model, we need to convert a sequence of tokens to a sequence of IDs. The `prepare_example` function below takes care of this for us. We then use these IDs as indices for the word embedding table. 

In [78]:
def prepare_example(example, vocab, shuffle = False):
    """
    Map tokens to their IDs for a single example
    """
  
      # vocab returns 0 if the word is not there (i2w[0] = <unk>)
    x = [vocab.w2i.get(t, 0) for t in example.tokens]
  
    x = torch.LongTensor([x])
    if shuffle:
        x =  x[:,torch.randperm(x.size()[1])]
    x = x.to(device)
  
    y = torch.LongTensor([example.label])
    y = y.to(device)
  
    return x, y

In [79]:
x, y = prepare_example(dev_data[0], v, shuffle = True)
print('x:', x)
print('y:', y)

x: tensor([[    8,   788, 19497,     3,  1895,    21,   606,    25,  1895,    62,
          3939,    20,     5]], device='cuda:0')
y: tensor([3], device='cuda:0')


#### Evaluation
We now need to define an evaluation metric.
How many predictions do we get right? The accuracy will tell us.
Make sure that you understand this code block.


In [80]:
def simple_evaluate(model, data, prep_fn=prepare_example, **kwargs):
  """Accuracy of a model on given data set."""
  correct = 0
  total = 0
  model.eval()  # disable dropout (explained later)

  for example in data:
    
    # convert the example input and label to PyTorch tensors
    x, target = prep_fn(example, model.vocab)

    # forward pass without backpropagation (no_grad)
    # get the output from the neural network for input x
    with torch.no_grad():
      logits = model(x)
    
    # get the prediction
    prediction = logits.argmax(dim=-1)
    
    # add the number of correct predictions to the total correct
    correct += (prediction == target).sum().item()
    total += 1

  return correct, total, correct / float(total)

We are using accuracy as a handy evaluation metric. Please consider using [alternative metrics](https://scikit-learn.org/stable/modules/classes.html#classification-metrics) for your experiments if that makes more theoretical sense (see, e.g., Q3.3 in Practical 1). 

#### Example feed
For stochastic gradient descent (SGD) we will need a random training example for every update.
We implement this by shuffling the training data and returning examples one by one using `yield`.

Shuffling is optional so that we get to use this function to get validation and test examples, too.

In [81]:
def get_examples(data, shuffle=True, **kwargs):
  """Shuffle data set and return 1 example at a time (until nothing left)"""
  if shuffle:
    print("Shuffling training data")
    random.shuffle(data)  # shuffle training data each epoch
  for example in data:
    yield example

#### Exercise: Training function

Your task is now to complete the training loop below.
Before you do so, please read the section about optimisation.

**Optimisation**

As mentioned in the "Intro to PyTorch" notebook, one of the perks of using PyTorch is automatic differentiation. We will use it to train our BOW model. 

We train our model by feeding it an input, performing a **forward** pass, obtaining an output prediction, and calculating a **loss** with our loss function.
After the gradients are computed in the **backward** pass, we can take a step on the surface of the loss function towards more optimal parameter settings (gradient descent). 

The package we will use to do this optimisation is [torch.optim](https://pytorch.org/docs/stable/optim.html). 

In [82]:
from torch import optim

Besides implementations of stochastic gradient descent (SGD), this package also implements the optimisation algorithm Adam, which we'll be using in this practical. 
For the purposes of this assignment you do not need to know what Adam does besides that it uses gradient information to update our model parameters by calling: 

```
optimizer.step()
```
Remember when we updated our parameters in the PyTorch tutorial in a loop?


```python
# update weights
learning_rate = 0.5
for f in net.parameters():
    # for each parameter, take a small step in the opposite dir of the gradient
    p.data = p.data - p.grad.data * learning_rate

```
The function call optimizer.step() does effectively the same thing.

*(If you want to know more about optimisation algorithms using gradient information, [this blog](http://ruder.io/optimizing-gradient-descent/.) gives a nice intuitive overview.)*

In [153]:
def train_model(model, optimizer, num_iterations=10000, 
                print_every=1000, eval_every=1000,
                batch_fn=get_examples, 
                prep_fn=prepare_example,
                eval_fn=simple_evaluate,
                batch_size=1, eval_batch_size=None, Shuffle=False, OnlyTest=False, data = train_data):
    """Train a model."""  
    #print("SHUFFLE", Shuffle)
    iter_i = 0
    train_loss = 0.
    print_num = 0
    start = time.time()
    criterion = nn.CrossEntropyLoss() # loss function
    best_eval = 0.
    best_iter = 0

    # store train loss and validation accuracy during training
    # so we can plot them afterwards
    losses = []
    accuracies = []  
    
    print("DDDAAATAAHAHAHA", len(data))

    if eval_batch_size is None:
        eval_batch_size = batch_size

    while True:  # when we run out of examples, shuffle and continue
        for batch in batch_fn(data, batch_size=batch_size):

            # forward pass
            model.train()
            #print("HALLOOOOO")
            x, targets = prep_fn(batch, model.vocab, Shuffle)
            #print("IN HET MODEL",x)
            logits = model(x)

            B = targets.size(0)  # later we will use B examples per update

            # compute cross-entropy loss (our criterion)
            # note that the cross entropy loss function computes the softmax for us
            loss = criterion(logits.view([B, -1]), targets.view(-1))
            train_loss += loss.item()

            # backward pass (tip: check the Introduction to PyTorch notebook)

            # erase previous gradients
            optimizer.zero_grad()
            # compute gradients
            loss.backward()
            # update weights - take a small step in the opposite dir of the gradient
            optimizer.step()

            print_num += 1
            iter_i += 1

            # print info
            if iter_i % print_every == 0:
                print("Iter %r: loss=%.4f, time=%.2fs" % 
                      (iter_i, train_loss, time.time()-start))
                losses.append(train_loss)
                print_num = 0        
                train_loss = 0.

            # evaluate
            if iter_i % eval_every == 0:
                _, _, accuracy = eval_fn(model, dev_data, batch_size=eval_batch_size,
                                         batch_fn=batch_fn, prep_fn=prep_fn)
                accuracies.append(accuracy)
                print("iter %r: dev acc=%.4f" % (iter_i, accuracy))       

                # save best model parameters
                if accuracy > best_eval:
                    print("new highscore")
                    best_eval = accuracy
                    best_iter = iter_i
                    path = "{}.pt".format(model.__class__.__name__)
                    ckpt = {
                      "state_dict": model.state_dict(),
                      "optimizer_state_dict": optimizer.state_dict(),
                      "best_eval": best_eval,
                      "best_iter": best_iter
                    }
                    torch.save(ckpt, path)

            # done training
            if iter_i == num_iterations:
                print("Done training")

                # evaluate on train, dev, and test with best model
                print("Loading best model")
                path = "{}.pt".format(model.__class__.__name__)        
                ckpt = torch.load(path)
                model.load_state_dict(ckpt["state_dict"])

#                 _, _, train_acc = eval_fn(
#                     model, train_data, batch_size=eval_batch_size, 
#                     batch_fn=batch_fn, prep_fn=prep_fn)
#                 _, _, dev_acc = eval_fn(
#                     model, dev_data, batch_size=eval_batch_size,
#                     batch_fn=batch_fn, prep_fn=prep_fn)
                _, _, test_acc = eval_fn(
                    model, test_data, batch_size=eval_batch_size, 
                    batch_fn=batch_fn, prep_fn=prep_fn)

#                 print("best model iter {:d}: "
#                       "train acc={:.4f}, dev acc={:.4f}, test acc={:.4f}".format(
#                           best_iter, train_acc, dev_acc, test_acc))
                if OnlyTest:
                    return test_acc

                return losses, accuracies

### Training the BOW model

In [84]:
# If everything is in place we can now train our first model!
bow_model = BOW(len(v.w2i), len(t2i), vocab=v)
print(bow_model)

bow_model = bow_model.to(device)

optimizer = optim.Adam(bow_model.parameters(), lr=0.0005)
bow_losses, bow_accuracies = train_model(
    bow_model, optimizer, num_iterations=200000, 
    print_every=1000, eval_every=1000)

BOW(
  (embed): Embedding(20727, 5)
)
Shuffling training data
Iter 1000: loss=5530.2435, time=0.95s
iter 1000: dev acc=0.1998
new highscore
Iter 2000: loss=5278.7102, time=2.18s
iter 2000: dev acc=0.2016
new highscore
Iter 3000: loss=4911.6843, time=3.41s
iter 3000: dev acc=0.2044
new highscore


KeyboardInterrupt: 

In [ ]:
# This will plot the validation accuracies across time.
plt.plot(bow_accuracies)

In [ ]:
# This will plot the training loss over time.
plt.plot(bow_losses)

Please note that we set the number of iterations to 30K as an indicative value, after which we simply stop training without checking for convergence. You should choose an appropriate number of iterations and motivate your decision. **This holds for all pre-set numbers of iteration in the following code blocks.**

# CBOW

We now continue with a **continuous bag-of-words (CBOW)** model. (*This is not the same as the word2vec CBOW model!*)

It is similar to the BOW model above, but now embeddings can have a dimension of *arbitrary size*. 
This means that we can choose a higher dimensionality and learn more aspects of each word. We will still sum word vectors to get a sentence representation, but now the size of the resulting vector will no longer correspond to the number of sentiment classes. 

So to turn the size of our summed vector into the number of output classes, we can *learn* a parameter matrix $W$ and multiply it by the sum vector $x$: $$Wx$$
If the size of $x$ is `d x 1`, we can set $W$ to be `5 x d`, so that the output of the matrix multiplication will be the of the desired size, `5 x 1`. Then, just like for the BOW model, we can obtain a prediction using the argmax function.

## Exercise: implement and train the CBOW model

Write a class `CBOW` that:

- has word embeddings with size 300
- sums the word vectors for the input words (just like in `BOW`)
- projects the resulting vector down to 5 units using a linear layer and a bias term (check out `nn.Linear`)

Train your CBOW model and plot the validation accuracy and training loss over time.

In [56]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, vocab):
        super(CBOW, self).__init__()
        self.vocab = vocab
        #self.w = torch.tensor((5,embedding_dim), requires_grad =True)
        
        # this is a trainable look-up table with word embeddings
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        
        self.linear = nn.Linear(embedding_dim,5)
        # this is a trainable bias term
        self.bias = nn.Parameter(torch.zeros(embedding_dim), requires_grad=True)        

    def forward(self, inputs):
        # this is the forward pass of the neural network
        # it applies a function to the input and returns the output

        # this looks up the embeddings for each word ID in inputs
        # the result is a sequence of word embeddings
        embeds = self.embed(inputs)
        summ = embeds.sum(1) + self.bias
        linear = self.linear(summ)
        #print(linear.size())
        #print(linear)
        
        # the output is the sum across the time dimension (1)
        # with the bias term added
       # logits = linear.sum(1) 
        #+ self.bias
        logits = linear

        return logits

In [57]:
cbow_model = CBOW(len(v.w2i), 300, vocab=v)
print(cbow_model)

cbow_model = cbow_model.to(device)

optimizer = optim.Adam(cbow_model.parameters(), lr=0.0005)
cbow_losses, cbow_accuracies = train_model(
    cbow_model, optimizer, num_iterations=100000, 
    print_every=1000, eval_every=1000)

CBOW(
  (embed): Embedding(20727, 300)
  (linear): Linear(in_features=300, out_features=5, bias=True)
)


NameError: name 'train_data' is not defined

In [58]:
# This will plot the validation accuracies across time.
plt.plot(cbow_accuracies)

NameError: name 'cbow_accuracies' is not defined

In [34]:
# This will plot the training loss over time.
plt.plot(cbow_losses)

NameError: name 'cbow_losses' is not defined

# Deep CBOW

To see if we can squeeze some more performance out of the CBOW model, we can make it deeper and non-linear by adding more layers and, e.g., tanh-activations.
By using more parameters we can learn more aspects of the data, and by using more layers and non-linearities, we can try to learn a more complex function. 
This is not something that always works. If the input-output mapping of your data is simple, then a complicated function could easily overfit on your training set, thereby leading to poor generalization. 

#### Exercise: write Deep CBOW class and train it

Write a class `DeepCBOW`.

In your code, make sure that your `output_layer` consists of the following:
- A linear transformation from E units to D units.
- A Tanh activation
- A linear transformation from D units to D units
- A Tanh activation
- A linear transformation from D units to 5 units (our output classes).

E is the size of the word embeddings (please use E=300) and D for the size of a hidden layer (please use D=100).

We recommend using [nn.Sequential](https://pytorch.org/docs/stable/nn.html?highlight=sequential#torch.nn.Sequential) to implement this exercise.

In [35]:
class DCBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, vocab,hidden_units):
        super(DCBOW, self).__init__()
        self.vocab = vocab
        
        # this is a trainable look-up table with word embeddings
        layers = []
        layers.append(nn.Embedding(vocab_size, embedding_dim))
        layers.append(nn.Linear(embedding_dim,hidden_units))
        layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_units,hidden_units))
        layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_units,5))
        
        self.layers=nn.Sequential(*layers)

    def forward(self, inputs):
        # this is the forward pass of the neural network
        # it applies a function to the input and returns the output

        # this looks up the embeddings for each word ID in inputs
        # the result is a sequence of word embeddings
        
        logits = self.layers(inputs)
        logits = logits.sum(1)
        return logits


In [36]:
dcbow_model = DCBOW(len(v.w2i), 300, vocab=v, hidden_units=100)
print(dcbow_model)

dcbow_model = dcbow_model.to(device)

optimizer = optim.Adam(dcbow_model.parameters(), lr=0.0005)
dcbow_losses, dcbow_accuracies = train_model(
    dcbow_model, optimizer, num_iterations=100000, 
    print_every=1000, eval_every=1000)

DCBOW(
  (layers): Sequential(
    (0): Embedding(0, 300)
    (1): Linear(in_features=300, out_features=100, bias=True)
    (2): Tanh()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): Tanh()
    (5): Linear(in_features=100, out_features=5, bias=True)
  )
)


NameError: name 'train_data' is not defined

In [37]:
# This will plot the validation accuracies across time.
plt.plot(dcbow_accuracies)

NameError: name 'dcbow_accuracies' is not defined

In [ ]:
# This will plot the training loss over time.
plt.plot(cbow_losses)

# Pre-trained word embeddings

The Stanford Sentiment Treebank is a rather small data set, since it required fine-grained manual annotatation. This makes it difficult for the Deep CBOW model to learn good word embeddings, i.e. to learn good word representations for the words in our vocabulary.
In fact, the only error signal that the network receives is from predicting the sentiment of entire sentences!

To start off with better word representations, we can download **pre-trained word embeddings**. 
You can choose which pre-trained word embeddings to use:

- **GloVe**. The "original" Stanford Sentiment classification [paper](http://aclweb.org/anthology/P/P15/P15-1150.pdf) used Glove embeddings, which are just another method (like *word2vec*) to get word embeddings from unannotated text. Glove is described in the following paper which you should cite if you use them:
> Jeffrey Pennington, Richard Socher, and Christopher Manning. ["Glove: Global vectors for word representation."](https://nlp.stanford.edu/pubs/glove.pdf) EMNLP 2014. 

- **Word2Vec**. This is the method that you learned about in class, described in:
> Mikolov, Tomas, et al. ["Distributed representations of words and phrases and their compositionality."](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) Advances in neural information processing systems. 2013.

Using these pre-trained word embeddings, we can initialize our word embedding lookup table and start form a point where similar words are already close to one another in the distributional semantic space. 

You can choose to keep the word embeddings **fixed** or to train them further, specialising them to the task at hand.
We will keep them fixed for now.

For the purposes of this lab, it is enough if you understand how word2vec works (whichever vectors you use), but if you are interested, we encourage you to also check out the GloVe paper.

You can either download the word2vec vectors, or the Glove vectors.
If you want to compare your results to the Stanford paper later on, then you should use Glove. 
**At the end of this lab you have the option to compare which vectors give you the best performance. For now, simply choose one of them and continue with that.**

[**OPTIONAL in case you don't want to mount Google Drive:** instead of running all the 5 boxes below, you can 1) download the GloVe and word2vec in your local machine, 2) upload them on your Drive folder ("My Drive"). Then, uncomment the first 2 lines in box 6 before writing your code!]


In [38]:
# This downloads the Glove 840B 300d embeddings.
# The original file is at http://nlp.stanford.edu/data/glove.840B.300d.zip
# Since that file is 2GB, we provide you with a *filtered version*
# which contains all the words you need for this data set.

# You only need to do this once.
# Please comment this cell out after downloading.

!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2020-12-04 10:06:28--  https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
Herleiden van gist.githubusercontent.com... 151.101.36.133
Verbinding maken met gist.githubusercontent.com|151.101.36.133|:443... verbonden.
FOUT: Kan certificaat van gist.githubusercontent.com (uitgegeven door /C=US/O=DigiCert Inc/OU=www.digicert.com/CN=DigiCert SHA2 High Assurance Server CA) niet controleren:
  Kan de autoriteit van de uitgever niet lokaal verifiëren.
FOUT: Naam 'www.github.com' in certificaat komt niet overeen met gevraagde hostnaam 'gist.githubusercontent.com'.
Gebruik '--no-check-certificate' om een onbeveiligde verbinding met gist.githubusercontent.com te maken.
Kan geen SSL-verbinding maken.


In [39]:
# This downloads the word2vec 300D Google News vectors 
# The file has been truncated to only contain words that appear in our data set.
# You can find the original file here: https://code.google.com/archive/p/word2vec/

# You only need to do this once.
# Please comment this out after downloading.
!wget https://gist.githubusercontent.com/bastings/4d1c346c68969b95f2c34cfbc00ba0a0/raw/76b4fefc9ef635a79d0d8002522543bc53ca2683/googlenews.word2vec.300d.txt

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2020-12-04 10:06:29--  https://gist.githubusercontent.com/bastings/4d1c346c68969b95f2c34cfbc00ba0a0/raw/76b4fefc9ef635a79d0d8002522543bc53ca2683/googlenews.word2vec.300d.txt
Herleiden van gist.githubusercontent.com... 151.101.36.133
Verbinding maken met gist.githubusercontent.com|151.101.36.133|:443... verbonden.
FOUT: Kan certificaat van gist.githubusercontent.com (uitgegeven door /C=US/O=DigiCert Inc/OU=www.digicert.com/CN=DigiCert SHA2 High Assurance Server CA) niet controleren:
  Kan de autoriteit van de uitgever niet lokaal verifiëren.
FOUT: Naam 'www.github.com' in certificaat komt niet overeen met gevraagde hostnaam 'gist.githubusercontent.com'.
Gebruik '--no-check-certificate' om een onbeveiligde verbinding met gist.githubusercontent.com te maken.
Kan geen SSL-verbinding maken.


In [40]:
# Mount Google Drive (to save the downloaded files)
from google.colab import drive
drive.mount('/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Copy word vectors *to* Google Drive

# You only need to do this once.
# Please comment this out after running it. 
!cp "glove.840B.300d.sst.txt" "/gdrive/My Drive/"
!cp "googlenews.word2vec.300d.txt" "/gdrive/My Drive/"

In [ ]:
# If you copied the word vectors to your Drive before,
# here is where you copy them back to the Colab notebook.

# Copy Glove vectors *from* Google Drive
!cp "/gdrive/My Drive/glove.840B.300d.sst.txt" .
!cp "/gdrive/My Drive/googlenews.word2vec.300d.txt" .

In [41]:
# Uncomment these 2 lines below if went for the OPTIONAL method described above
# !cp "glove.840B.300d.sst.txt" "./"
# !cp "googlenews.word2vec.300d.txt" "./"

At this point you have the pre-trained word embedding files, but what do they look like?

In [85]:
with open("./glove.840B.300d.sst.txt", mode="r", encoding="utf-8") as f:
    for x in f:
        print(len(x.split(' ')))
        print(x)
        break
    

301
, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 

#### Exercise: New Vocabulary

Since we now use pre-trained word embeddings, we need to create a new vocabulary. 
This is because of two reasons:

1. We do not have pre-trained word embeddings for all words in our SST training set, and we do not want words in our vocabulary for which we have no word embeddings.
2. We should be able to look up the pre-trained word embedding for words in the validation and test set, even if these words are unseen in training. 

Now, create a new vocabulary object `v` based on the word set of pre-trained embeddings, and load the corresponding embeddings into a list `vectors`.

The vocabulary `v` should consist of:
 - a  `<unk>` token at position 0,
 - a  `<pad>` token at position 1, 
 - and then all words in the pre-trained embedding set.
 

After storing each vector in a list `vectors`, turn the list into a numpy matrix like this:
```python
 vectors = np.stack(vectors, axis=0)
```

Remember to add new embeddings for the `<unk>` and `<pad>` tokens, as they're not part of the word2vec/GloVe embeddings. These embeddings can be randomly initialized or 0-valued, think about what makes sense and see what the effects are.
 

In [86]:
# YOUR CODE HERE
v = Vocabulary()
print(v.w2i)
print(v.i2w)
vectors = []
with open("./glove.840B.300d.sst.txt", mode="r", encoding="utf-8") as f:
    for x in f:
     
        split = x.split(' ')
       
        word = split[0]
#         print(word)
        
        split = split[1:]
        
        v.count_token(word)
        #vv.add_token(word)
        
        
        embedding = np.zeros(300)

        i = 0
        for element in split:
            embedding[i] = float(element)
            i += 1
        vectors.append(embedding)
    

vectors.insert(0,np.zeros(300))
vectors.insert(0,np.zeros(300))
vectors = np.stack(vectors, axis = 0)
v.build()




# This process should be deterministic and should have the same result 
# if run multiple times on the same data set.

{}
[]


In [165]:
with open("./glove.840B.300d.sst.txt", mode="r", encoding="utf-8") as f:
    for x in f:
        print(x)
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [167]:
print(v.w2i.keys())
# print(v.w2i['of'])
# print(v.w2i['.'])
# v.i2w[:10]
# print(vectors[0])
# print(vectors[1])
# print(vectors[2])


dict_keys(['<unk>', '<pad>', ',', '.', 'the', 'and', 'to', 'of', 'a', 'in', ':', 'is', 'for', 'I', ')', '(', 'that', '-', 'on', 'you', 'with', "'s", 'it', 'The', 'are', 'by', 'at', 'be', 'this', 'as', 'from', 'was', 'have', 'or', '...', 'your', 'not', '!', '?', 'will', 'an', "n't", 'can', 'but', 'all', 'my', 'has', 'do', 'we', 'they', 'more', 'one', 'about', 'he', ';', "'", 'out', '$', 'their', 'so', 'his', 'up', 'It', '&', 'like', '/', '1', 'which', 'if', 'would', 'our', 'me', 'who', 'just', 'This', 'time', 'what', 'A', '2', 'had', 'when', 'there', 'been', 'some', 'get', 'were', 'other', 'also', 'In', 'her', 'them', 'You', 'new', 'We', 'no', 'any', 'people', 'than', 'into', 'only', '3', 'how', 'its', 'first', 'said', 'If', 'over', 'make', 'good', 'know', 'very', '%', 'am', 'now', 'see', 'may', 'she', 'could', 'most', 'then', "'m", 'use', 'these', 'did', 'And', 'way', 'New', '4', 'here', 'well', 'work', 'two', '=', 'think', 'much', 'should', 'us', 'years', 'many', 'back', 'because', '5

In [47]:
# print(vectors[2, :])
print(vectors.shape)

(20727, 300)


#### Exercise: words not in our pre-trained set

How many words in the training, dev, and test set are also in your vector set?
How many words are not there?

Store the words that are not in the word vector set in the set below.

In [88]:
words_not_found = set()

for data_set in [train_data, test_data, dev_data]:
    
    for ex in data_set:
            for token in ex.tokens:
                
                # check if token is not in the vocab
                if not token in v.w2i:
                    words_not_found.add(token)
            
            
           
print(words_not_found)
len(words_not_found)
# YOUR CODE HERE

{'Clarke-Williams', 'Reeboir', 'grade-grubbers', 'media-soaked', 'Oprahfication', 'Hubac', 'non-Shakespeare', 'disease-of-the-week', 'autocritique', 'cliche-bound', 'Thing-type', 'tough-man', 'over-romanticize', 'matinee-style', 'Cool-J', 'semi-amusing', '91-minute', 'blue-light-special', 'alientation', 'well-contructed', 'roisterous', 'Co-writer/director', 'Holofcenter', 'Stevenon', 'green-guts', 'tear-drenched', 'difficult-to-swallow', 'Auteil', 'doofus-on', 'LePlouff', 'beat-charged', 'Jae-eun', 'oh-those-wacky-Brits', 'guy-in-a-dress', 'sense-of-humour', 'indieflick', 'clung-to', 'heart-affecting', 'screeching-metal', 'singer-turned', 'too-spectacular', 'life-at-arm', 'so-five-minutes-ago', 'daytime-drama', 'diciness', '179-minute', 'surfer-girl', 'modern-office', 'not-so-Divine', 'Pie-type', 'follow-your-dream', 'food-spittingly', 'goose-pimple', 'anti-Kieslowski', 'pokepie', 'pseudo-rock-video', 'jump-in-your-seat', 'trance-noir', 'accordion/harmonica/banjo', 'Glizty', 'road-and-

976

#### Exercise: train Deep CBOW with (fixed) pre-trained embeddings

Now train Deep CBOW again using the pre-trained word vectors.


In [ ]:
class PTDeepCBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, vocab, embedding_w):

#         super(PTDeepCBOW, self).__init__(vocab_size, embedding_dim, hidden_dim, output_dim, vocab, embedding_w)
        super(PTDeepCBOW, self).__init__()
        self.vocab = vocab
    
        embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        embedding_layer.weights = embedding_w
        embedding_layer.weight.requires_grad = False
        
        hidden_units = hidden_dim
        
# # disable training the pre-trained embeddings
# pt_deep_cbow_model.embed.weight.requires_grad = False

        
        # this is a trainable look-up table with word embeddings
        layers = []
        layers.append(embedding_layer)
        


        layers.append(nn.Linear(embedding_dim,hidden_units))
        layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_units,hidden_units))
        layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_units,5))
        
        self.layers=nn.Sequential(*layers)

    def forward(self, inputs):
        # this is the forward pass of the neural network
        # it applies a function to the input and returns the output

        # this looks up the embeddings for each word ID in inputs
        # the result is a sequence of word embeddings
        
        # word >>> word 2 i 
        #index >> embeddings
        
        logits = self.layers(inputs)
        logits = logits.sum(1)
        return logits


In [49]:
# Create a Deep CBOW model with pre-trained embeddings
# YOUR CODE HERE
# pt_deep_cbow_model = ..




# # copy pre-trained word vectors into embeddings table
# pt_deep_cbow_model.embed.weight.data.copy_(torch.from_numpy(vectors))


embed_w = torch.from_numpy(vectors)



pt_deep_cbow_model = PTDeepCBOW( len(v.w2i), 300, hidden_dim=100, output_dim = 5, vocab=v,
                                embedding_w = embed_w)

# move model to specified device
pt_deep_cbow_model = pt_deep_cbow_model.to(device)
# train the model



optimizer = optim.Adam(pt_deep_cbow_model.parameters(), lr=0.0005)

PTDbow_losses, PTDbow_accuracies = train_model(
    pt_deep_cbow_model, optimizer, num_iterations=10000, 
    print_every=1000, eval_every=1000)

# YOUR CODE HERE

NameError: name 'PTDeepCBOW' is not defined

In [ ]:
# plot dev accuracies

In [ ]:
# plot train loss

**It looks like we've hit what is possible with just using words.**
Let's move on by incorporating word order!

# LSTM

It is time to get more serious. Even with pre-trained word embeddings and multiple layers, we still seem to do pretty badly at sentiment classification. 
The next step we can take is to introduce word order again, dropping our independence assumptions. In this way, we can get a representation of the sentence as an ordered set of tokens.

We will get this representation using a **Long Short-Term Memory** (LSTM). As an exercise, we will code our own LSTM cell, so that we get comfortable with its inner workings.
Once we have an LSTM cell, we can call it repeatedly, updating its hidden state one word at a time:

```python
rnn = MyLSTMCell(input_size, hidden_size)

hx = torch.zeros(1, hidden_size)  # initial hidden state
cx = torch.zeros(1, hidden_size)  # initial memory cell
output = []                       # to save intermediate LSTM states

# feed one word at a time
for i in range(n_timesteps):
  hx, cx = rnn(input[i], (hx, cx))
  output.append(hx)
```

If you need some more help understanding LSTMs, you can check out these resources:
- Blog post (highly recommended): http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Paper covering LSTM formulas in detail: https://arxiv.org/abs/1503.04069 


#### Exercise: Finish the LSTM cell below. 
You will need to implement the LSTM formulas:

$$
\begin{array}{ll}
        i = \sigma(W_{ii} x + b_{ii} + W_{hi} h + b_{hi}) \\
        f = \sigma(W_{if} x + b_{if} + W_{hf} h + b_{hf}) \\
        g = \tanh(W_{ig} x + b_{ig} + W_{hg} h + b_{hg}) \\
        o = \sigma(W_{io} x + b_{io} + W_{ho} h + b_{ho}) \\
        c' = f * c + i * g \\
        h' = o \tanh(c') \\
\end{array}
 $$

where $\sigma$ is the sigmoid function.

*Note that the LSTM formulas can differ slightly between different papers. We use the PyTorch LSTM formulation here.*

In [89]:
class MyLSTMCell(nn.Module):
  """Our own LSTM cell"""

  def __init__(self, input_size, hidden_size, bias=True):
    """Creates the weights for this LSTM"""
    super(MyLSTMCell, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size    
    #print( self.input_size )
#print(self.hidden_size)
       
    self.W_ii = nn.Parameter(torch.empty((self.hidden_size, self.input_size)))
    self.W_hi = nn.Parameter(torch.empty((self.hidden_size, self.hidden_size)))
    self.B_ii = nn.Parameter(torch.empty(self.hidden_size))
    self.B_hi = nn.Parameter(torch.empty(self.hidden_size))
    
    self.W_if = nn.Parameter(torch.empty((self.hidden_size,self.input_size)))
    self.W_hf = nn.Parameter(torch.empty((self.hidden_size, self.hidden_size)))
    self.B_if = nn.Parameter(torch.empty(self.hidden_size))
    self.B_hf = nn.Parameter(torch.empty(self.hidden_size))

    self.W_ig = nn.Parameter(torch.empty((self.hidden_size, self.input_size)))
    self.W_hg = nn.Parameter(torch.empty((self.hidden_size, self.hidden_size)))
    self.B_ig = nn.Parameter(torch.empty(self.hidden_size))
    self.B_hg = nn.Parameter(torch.empty(self.hidden_size))

    self.W_io = nn.Parameter(torch.empty((self.hidden_size, self.input_size)))
    self.W_ho = nn.Parameter(torch.empty((self.hidden_size, self.hidden_size)))
    self.B_io = nn.Parameter(torch.empty(self.hidden_size))
    self.B_ho = nn.Parameter(torch.empty(self.hidden_size))


    self.reset_parameters()

  def reset_parameters(self):
    """This is PyTorch's default initialization method"""
    
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
        weight.data.uniform_(-stdv, stdv)  

  def forward(self, input_, hx, mask=None):
    """
    input is (batch, input_size)
    hx is ((batch, hidden_size), (batch, hidden_size))
    """
    
    prev_h, prev_c = hx
    

    # project input and prev state

    # main LSTM computation   
    
    #print(torch.mean(self.W_ii))
    i = torch.sigmoid(input_ @ torch.transpose(self.W_ii,0,1) + self.B_ii +prev_h @ self.W_hi + self.B_hi)
    f = torch.sigmoid(input_ @ torch.transpose(self.W_if,0,1) + self.B_if + prev_h @ self.W_hf + self.B_hf)
    g = torch.tanh(input_ @ torch.transpose(self.W_ig,0,1) + self.B_ig + prev_h@ self.W_hg + self.B_hg)
    o = torch.sigmoid(input_@ torch.transpose(self.W_io ,0,1) + self.B_io + prev_h @ self.W_ho + self.B_ho)
    #print(i,f,g,o)
    
  #  print("H", prev_h.size())
  #  print(f.size())
##  print("OLD",prev_c.size() )
   # print(i.size())

    c = f * prev_c + i * g
   # print(c.size())
    h = o * torch.tanh(c)


    return h, c
  
  def __repr__(self):
    return "{}({:d}, {:d})".format(
        self.__class__.__name__, self.input_size, self.hidden_size)

#### Optional: Efficient Matrix Multiplication

It is more efficient to do a few big matrix multiplications than to do many smaller ones. So we will implement the above cell using just **two** linear layers.

This is possible because the eight linear transformations contained in one forward pass through an LSTM cell can be reduced to just two:
$$W_h h + b_h$$
$$W_i x + b_i $$ 

with $h = $ `prev_h` and $x = $ `input_`.

and where: 

$W_h =  \begin{pmatrix}
W_{hi}\\ 
W_{hf}\\ 
W_{hg}\\ 
W_{ho}
\end{pmatrix}$, $b_h = \begin{pmatrix}
b_{hi}\\ 
b_{hf}\\ 
b_{hg}\\ 
b_{ho}
\end{pmatrix}$,  $W_i = \begin{pmatrix}
W_{ii}\\ 
W_{if}\\ 
W_{ig}\\ 
W_{io}
\end{pmatrix}$ and $b_i = \begin{pmatrix}
b_{ii}\\ 
b_{if}\\ 
b_{ig}\\ 
b_{io}
\end{pmatrix}$.

Convince yourself that, after chunking with [torch.chunk](https://pytorch.org/docs/stable/torch.html?highlight=chunk#torch.chunk), the output of those two linear transformations is equivalent to the output of the eight linear transformations in the LSTM cell calculations above.

#### LSTM Classifier

Having an LSTM cell is not enough: we still need some code that calls it repeatedly, and then makes a prediction from the final hidden state. 
You will find that code below. Make sure that you understand it.

In [93]:
class LSTMClassifier(nn.Module):
  """Encodes sentence with an LSTM and projects final hidden state"""

  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, vocab):
    super(LSTMClassifier, self).__init__()
    self.vocab = vocab
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
    self.rnn = MyLSTMCell(embedding_dim, hidden_dim)
    
    self.output_layer = nn.Sequential(     
        nn.Dropout(p=0.5),  # explained later
        nn.Linear(hidden_dim, output_dim)
    )

  def forward(self, x):
    
    B = x.size(0)  # batch size (this is 1 for now, i.e. 1 single example)
    T = x.size(1)  # timesteps (the number of words in the sentence)
    #print(x)
    input_ = self.embed(x)

    # here we create initial hidden states containing zeros
    # we use a trick here so that, if input is on the GPU, then so are hx and cx
    hx = input_.new_zeros(B, self.rnn.hidden_size)
    cx = input_.new_zeros(B, self.rnn.hidden_size)
    
    # process input sentences one word/timestep at a time
    # input is batch-major (i.e., batch size is the first dimension)
    # so the first word(s) is (are) input_[:, 0]
    outputs = []   
    for i in range(T):
        
        #sys.exit()
        hx, cx = self.rnn(input_[:, i], (hx, cx))
        outputs.append(hx)
    
    # if we have a single example, our final LSTM state is the last hx
    if B == 1:
        final = hx
    else:
      #
      # This part is explained in next section, ignore this else-block for now.
      #
      # We processed sentences with different lengths, so some of the sentences
      # had already finished and we have been adding padding inputs to hx.
      # We select the final state based on the length of each sentence.
      
      # two lines below not needed if using LSTM from pytorch
        outputs = torch.stack(outputs, dim=0)           # [T, B, D]
        outputs = outputs.transpose(0, 1).contiguous()  # [B, T, D]

        # to be super-sure we're not accidentally indexing the wrong state
        # we zero out positions that are invalid
        pad_positions = (x == 1).unsqueeze(-1)

        outputs = outputs.contiguous()      
        outputs = outputs.masked_fill_(pad_positions, 0.)

        mask = (x != 1)  # true for valid positions [B, T]
        lengths = mask.sum(dim=1)                 # [B, 1]

        indexes = (lengths - 1) + torch.arange(B, device=x.device, dtype=x.dtype) * T
        final = outputs.view(-1, self.hidden_dim)[indexes]  # [B, D]
    
    # we use the last hidden state to classify the sentence
    logits = self.output_layer(final)
    return logits

#### Dropout

Data sparsity and a small data set can cause *overfitting*. This is a phenomenom that is very likely to occur when training strong and expressive models, like LSTMs, on small data. In practice, if your model overfits, this means that it will be very good at predicting (or "remembering") the sentiment of the training set, but unable to generalise to new, unseen data in the test set. This is undesirable and one technique to mitigate this issue is *dropout*. 

A dropout layer is defined by the following formula, which can be applied, for example, to a linear layer:

$$\text{tanh}(W(\mathbf{h}\odot \mathbf{d}) + \mathbf{b})$$

where $\mathbf{d} \in \{0, 1\}^n$, with $d_j \sim \text{Bernoulli}(p)$, 

These formula simply means that we *drop* certain parameters during training (by setting them to zero). Which parameters we drop is stochastically determined by a Bernoulli distribution and the probability of each parameter being dropped is set to $p = 0.5$ in our experiments (see the previous cell of code where we define our output layer). A dropout layer can be applied at many different places in our models. This technique helps against the undesirable effect that a model relies on single parameters for prediction (e.g. if $h^{\prime}_j$ is large, always predict positive). If we use dropout, the model needs to learn to rely on different parameters, which is desirable to obtain better generalisation to unseen data.

**Let's train our LSTM!** Note that is will be a lot slower than previous models because we need to do many more computations per sentence.

#### Training

In [94]:

print(len(vectors))
print(len(v.w2i))
print(len(t2i))

20727
20727
5


In [95]:
lstm_model = LSTMClassifier(len(v.w2i), 300, 168, len(t2i), v)

# copy pre-trained word vectors into embeddings table
print(len(vectors))

with torch.no_grad():
    lstm_model.embed.weight.data.copy_(torch.from_numpy(vectors))
    lstm_model.embed.weight.requires_grad = False

print(lstm_model)
print_parameters(lstm_model)

lstm_model = lstm_model.to(device)
optimizer = optim.Adam(lstm_model.parameters(), lr=3e-4)

lstm_losses, lstm_accuracies = train_model(
    lstm_model, optimizer, num_iterations=25000, 
    print_every=250, eval_every=1000)



20727
LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.W_ii                 [168, 300]   requires_grad=True
rnn.W_hi                 [168, 168]   requires_grad=True
rnn.B_ii                 [168]        requires_grad=True
rnn.B_hi                 [168]        requires_grad=True
rnn.W_if                 [168, 300]   requires_grad=True
rnn.W_hf                 [168, 168]   requires_grad=True
rnn.B_if                 [168]        requires_grad=True
rnn.B_hf                 [168]        requires_grad=True
rnn.W_ig                 [168, 300]   requires_grad=True
rnn.W_hg                 [168, 168]   requires_grad=True
rnn.B_ig                 [168]        requires_grad=True
rnn.B_hg                 [168]        requires_grad=True
rnn.W_io               

KeyboardInterrupt: 

In [ ]:
# plot validation accuracy
plt.plot(lstm_accuracies)

In [ ]:
# plot training loss
plt.plot(lstm_losses)

# Mini-batching



**Why is the LSTM so slow?** Despite our best efforts, we still need to make a lot of matrix multiplications per example (linear in the length of the example) just to get a single classification, and we can only process the 2nd word once we have computed the hidden state for the 1st word (sequential computation).

GPUs are more efficient if we do a few big matrix multiplications, rather than lots of small ones. If we could process multiple examples at the same time, then we could exploit that. That is, we could still process the input sequentially, but doing so for multiple sentences at the same time.

Up to now our "mini-batches" consisted of a single example. This was for a reason: the sentences in our data sets have **different lengths**, and this makes it difficult to process them at the same time.

Consider a batch of 2 sentences:

```
this movie is bad
this movie is super cool !
```

Let's say the IDs for these sentences are:

```
2 3 4 5
2 3 4 6 7 8
```

We cannot feed PyTorch an object with rows of variable length! We need to turn this into a matrix.

The solution is to add **padding values** to our mini-batch:

```
2 3 4 5 1 1
2 3 4 6 7 8
```

Whenever a sentence is shorter than the longest sentence in a mini-batch, we just use a padding value (here: 1) to fill the matrix.

In our computation, we should **ignore** the padding positions (e.g. mask them out) so that paddings do not contribute to the loss.

#### Mini-batch feed
We will now implement a `get_minibatch` function which will replace `get_example` and returns a mini-batch of the requested size.

In [96]:
def get_minibatch(data, batch_size=25, shuffle=True):
  """Return minibatches, optional shuffling"""
  
  if shuffle:
    print("Shuffling training data")
    random.shuffle(data)  # shuffle training data each epoch
  
  batch = []
  
  # yield minibatches
  for example in data:
    batch.append(example)
    
    if len(batch) == batch_size:
      yield batch
      batch = []
      
  # in case there is something left
  if len(batch) > 0:
    yield batch

#### Padding function
We will need a function that adds padding 1s to a sequence of IDs so that
it becomes as long as the longest sequence in the minibatch.

In [97]:
def pad(tokens, length, pad_value=1):
  """add padding 1s to a sequence to that it has the desired length"""
  return tokens + [pad_value] * (length - len(tokens))

# example
tokens = [2, 3, 4]
pad(tokens, 5)

[2, 3, 4, 1, 1]

#### New `prepare` function

We will also need a new function that turns a mini-batch into PyTorch tensors.

In [98]:
def prepare_minibatch(mb, vocab, Shuffle = False):
  """
  Minibatch is a list of examples.
  This function converts words to IDs and returns
  torch tensors to be used as input/targets.
  """
  batch_size = len(mb)
  maxlen = max([len(ex.tokens) for ex in mb])
  
  # vocab returns 0 if the word is not there
  x = [pad([vocab.w2i.get(t, 0) for t in ex.tokens], maxlen) for ex in mb]
  
  x = torch.LongTensor(x)
  x = x.to(device)
  
  y = [ex.label for ex in mb]
  y = torch.LongTensor(y)
  y = y.to(device)
  
  return x, y

In [99]:
# Let's test our new function.
# This should give us 3 examples.
mb = next(get_minibatch(train_data, batch_size=3, shuffle=False))
for ex in mb:
    None
#     print(ex)
#     print(ex.tokens)
#     print("-----------------")

In [100]:
# We should find padding 1s at the end
x, y = prepare_minibatch(mb, v)
print("x", x)
print("y", y)

x tensor([[12588, 17780,    21,  6232,    11, 15281,     2,    43,    22,    21,
           349,     6,   835,    16, 10406,     7, 13059,  8696,   739,    89,
         14051,  2586,     3,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1],
        [ 1835,   137,     7,  2379,   576,     5,  1479,  7399,    98,    40,
         17589,     2,  7813,     5, 11384,  2916,     7,  5317,     3,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1],
        [ 1009,    68,     4, 10595,  5335,     7,     8,   255,     7, 15531,
           544,  4295,     9,     8,   606,    52,  3030, 14480,   123,    41,
          7196,  6891,     4,  8353,     2, 12591, 14320,     

#### Evaluate (mini-batch version)

We can now update our evaluation function to use mini-batches

In [101]:
def evaluate(model, data, 
             batch_fn=get_minibatch, prep_fn=prepare_minibatch,
             batch_size=16):
  """Accuracy of a model on given data set (using mini-batches)"""
  correct = 0
  total = 0
  model.eval()  # disable dropout

  for mb in batch_fn(data, batch_size=batch_size, shuffle=False):
    x, targets = prep_fn(mb, model.vocab)
    with torch.no_grad():
      logits = model(x)
      
    predictions = logits.argmax(dim=-1).view(-1)
    
    # add the number of correct predictions to the total correct
    correct += (predictions == targets.view(-1)).sum().item()
    total += targets.size(0)

  return correct, total, correct / float(total)

# LSTM (Mini-batched)

With this, let's run the LSTM again but now using mini-batches!

In [102]:
lstm_model = LSTMClassifier(
    len(v.w2i), 300, 168, len(t2i), v)

# copy pre-trained vectors into embeddings table
with torch.no_grad():
  lstm_model.embed.weight.data.copy_(torch.from_numpy(vectors))
  lstm_model.embed.weight.requires_grad = False

print(lstm_model)
print_parameters(lstm_model)  
  
lstm_model = lstm_model.to(device)

batch_size = 25
optimizer = optim.Adam(lstm_model.parameters(), lr=2e-4)

lstm_losses, lstm_accuracies = train_model(
    lstm_model, optimizer, num_iterations=30000, 
    print_every=250, eval_every=250,
    batch_size=batch_size,
    batch_fn=get_minibatch, 
    prep_fn=prepare_minibatch,
    eval_fn=evaluate)

LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.W_ii                 [168, 300]   requires_grad=True
rnn.W_hi                 [168, 168]   requires_grad=True
rnn.B_ii                 [168]        requires_grad=True
rnn.B_hi                 [168]        requires_grad=True
rnn.W_if                 [168, 300]   requires_grad=True
rnn.W_hf                 [168, 168]   requires_grad=True
rnn.B_if                 [168]        requires_grad=True
rnn.B_hf                 [168]        requires_grad=True
rnn.W_ig                 [168, 300]   requires_grad=True
rnn.W_hg                 [168, 168]   requires_grad=True
rnn.B_ig                 [168]        requires_grad=True
rnn.B_hg                 [168]        requires_grad=True
rnn.W_io                 [168

KeyboardInterrupt: 

In [ ]:
# plot validation accuracy
plt.plot(lstm_accuracies)

In [ ]:
# plot training loss
plt.plot(lstm_losses)

# Tree-LSTM

In the final part of this lab we will exploit the tree structure of the SST data. 
Until now we only used the surface tokens, but remember that our data examples include binary trees with a sentiment score at every node.

In particular, we will implement **N-ary Tree-LSTMs** which are described in:

> Kai Sheng Tai, Richard Socher, and Christopher D. Manning. [Improved Semantic Representations From Tree-Structured Long Short-Term Memory Networks](http://aclweb.org/anthology/P/P15/P15-1150.pdf) ACL 2015.

Since our trees are binary (i.e., N=2), we can refer to these as *Binary Tree-LSTMs*. If you study equations (9) to (14) in the paper, you will find that they are not all too different from the original LSTM that you already have.

You should read this paper carefully and make sure that you understand the approach. You will also find our LSTM baseline there.
Note however that Tree-LSTMs were proposed around the same time by two other groups:

> Phong Le and Willem Zuidema. [Compositional distributional semantics with long short term memory](http://anthology.aclweb.org/S/S15/S15-1002.pdf). *SEM 2015.

> Xiaodan Zhu, Parinaz Sobihani,  and Hongyu Guo. [Long short-term memory over recursive structures](http://proceedings.mlr.press/v37/zhub15.pdf). ICML 2015.

It is good scientific practice to cite all three papers in your report.




## Computation

Do you remember the `transitions_from_treestring` function all the way in the beginning of this lab? Every example contains a **transition sequence** produced by this function. Let's look at it again:



In [103]:
ex = next(examplereader("trees/dev.txt"))
print(TreePrettyPrinter(ex.tree))
print("Transitions:")
print(ex.transitions)

print(len(ex.transitions))

              3                                                                     
  ____________|____________________                                                  
 |                                 4                                                
 |        _________________________|______________________________________________   
 |       4                                                                        | 
 |    ___|______________                                                          |  
 |   |                  4                                                         | 
 |   |         _________|__________                                               |  
 |   |        |                    3                                              | 
 |   |        |               _____|______________________                        |  
 |   |        |              |                            4                       | 
 |   |        |              |            ________________|_

Note that the tree is **binary**. Every node has two children, except for pre-terminal nodes.

A tree like this can be described by a sequence of **SHIFT (0)** and **REDUCE (1)** actions.

To construct a tree, we can use the transitions as follows:
- **reverse** the sentence (a list of tokens) and call this the **buffer**
   - the first word is now on top (last in the list), and we would get it when calling pop() on the buffer
- create an empty list and call it the **stack**
- iterate through the transition sequence:
  - if it says SHIFT(0), we pop a word from the buffer, and push it to the stack
  - if it says REDUCE(1), we pop the **top two items** from the stack, and combine them (e.g. with a Tree-LSTM!), creating a new node that we push back on the stack
  
Convince yourself that going through the transition sequence above will result in the tree that you see.
For example, we would start by putting the following words on the stack (by shifting 5 times, starting with `It`):

```
Top of the stack:
-----------------
film
lovely
a 
's  
It
```
Now we find a REDUCE in the transition sequence, so we get the top two words (film and lovely), and combine them, so our new stack becomes:
```
Top of the stack:
-----------------
lovely film
a 
's  
It
```

We will use this approach when encoding sentences with our Tree-LSTM.
Now, our sentence is a reversed list of word embeddings.
When we shift, we move a word embedding to the stack.
When we reduce, we apply the Tree-LSTM to the top two vectors, and the result is a single vector that we put back on the stack.
After going through the whole transition sequence, we will have the root node on our stack! We can use that to classify the sentence.



## Obtaining the transition sequence



So what goes on in the `transitions_from_treestring` function?

The idea ([explained in this blog post](https://devblogs.nvidia.com/recursive-neural-networks-pytorch/)) is that, if we had a tree, we could traverse through the tree, and every time that we find a node containing only a word, we output a SHIFT.
Every time **after** we have finished visiting the children of a node, we output a REDUCE.
(What is this tree traversal called?)

However, our `transitions_from_treestring` function operates directly on the string representation. It works as follows.

We start with the representation:

```
(3 (2 It) (4 (4 (2 's) (4 (3 (2 a) (4 (3 lovely) (2 film))) (3 (2 with) (4 (3 (3 lovely) (2 performances)) (2 (2 by) (2 (2 (2 Buy) (2 and)) (2 Accorsi))))))) (2 .)))
```

First we remove pre-terminal nodes (and add spaces before closing brackets):

```
(3 It (4 (4 's (4 (3 a (4 lovely film ) ) (3 with (4 (3 lovely performances ) (2 by (2 (2 Buy and )  Accorsi ) ) ) ) ) ) . ) )
```

Then we remove node labels:

```
( It ( ( 's ( ( a ( lovely film ) ) ( with ( ( lovely performances) ( by ( ( Buy and )  Accorsi ) ) ) ) ) ) . ) )
```

Then we remove opening brackets:

```
It 's a lovely film ) ) with lovely performances ) by Buy and ) Accorsi ) ) ) ) ) ) . ) )
```

Now we replace words by S (for SHIFT), and closing brackets by R (for REDUCE):

```
S S S S S R R S S S R S S S R S R R R R R R S R R
0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 
```

Et voilà. We just obtained the transition sequence!

In [104]:
# for comparison
seq = ex.transitions
s = " ".join(["S" if t == 0 else "R" for t in seq])
print(s)
print(" ".join(map(str, seq)))

S S S S S R R S S S R S S S R S R R R R R R S R R
0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1


## Coding the Tree-LSTM

The code below contains a Binary Tree-LSTM cell.
It is used in the TreeLSTM class below it, which in turn is used in the TreeLSTMClassifier.
The job of the TreeLSTM class is to encode a complete sentence and return the root node.
The job of the TreeLSTMCell is to return a new state when provided with two children (a reduce action). By repeatedly calling the TreeLSTMCell, the TreeLSTM will encode a sentence. This can be done for multiple sentences at the same time.


#### Exercise 
Check the `forward` function and complete the Tree-LSTM formulas.
You can see that we defined a large linear layer for you, that projects the *concatenation* of the left and right child into the input gate, left forget gate, right forget gate, candidate, and output gate.

In [105]:
class TreeLSTMCell(nn.Module):
  """A Binary Tree LSTM cell"""

  def __init__(self, input_size, hidden_size, bias=True):
    """Creates the weights for this LSTM"""
    super(TreeLSTMCell, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias

    self.reduce_layer = nn.Linear(2 * hidden_size, 5 * hidden_size)
    self.dropout_layer = nn.Dropout(p=0.25)

    self.reset_parameters()

  def reset_parameters(self):
    """This is PyTorch's default initialization method"""
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)  

  def forward(self, hx_l, hx_r, mask=None):
    """
    hx_l is ((batch, hidden_size), (batch, hidden_size))
    hx_r is ((batch, hidden_size), (batch, hidden_size))    
    """
    prev_h_l, prev_c_l = hx_l  # left child
    prev_h_r, prev_c_r = hx_r  # right child

    B = prev_h_l.size(0)

    # we concatenate the left and right children
    # you can also project from them separately and then sum
    children = torch.cat([prev_h_l, prev_h_r], dim=1)
    
    # project the combined children into a 5D tensor for i,fl,fr,u,o
    # this is done for speed, and you could also do it separately
    proj = self.reduce_layer(children)  # shape: B x 5D

    # each shape: B x D
    i, f_l, f_r, u, o = torch.chunk(proj, 5, dim=-1)

    # main Tree LSTM computation
    
    # YOUR CODE HERE
    # You only need to complete the commented lines below.
#     raise NotImplementedError("Implement this.")

    # The shape of each of these is [batch_size, hidden_size]
    


    i = torch.sigmoid(i)
    f_l = torch.sigmoid(f_l)    
    f_r = torch.sigmoid(f_r)
    u = torch.tanh(u)    
    o = torch.sigmoid(o)

    # j = hidden
    # k = memory
    c = i * u + f_l * prev_c_l  + f_r * prev_c_r
    h = o * torch.tanh(c)
    
    return h, c
  
  def __repr__(self):
    return "{}({:d}, {:d})".format(
        self.__class__.__name__, self.input_size, self.hidden_size)

## Explanation of the TreeLSTM class


The code below contains the TreeLSTM class, which implements everything we need in order to encode a sentence from word embeddings. The calculations are the same as in the paper, implemented such that the class `TreeLSTMCell` above is as general as possible and only takes two children to reduce them into a parent. 


**Initialize $\mathbf{h}$ and $\mathbf{c}$ outside of the cell for the leaves**

At the leaves of each tree the children nodes are **empty**, whereas in higher levels the nodes are binary tree nodes that *do* have a left and right child (but no input $x$). By initializing the leaf nodes outside of the cell class (`TreeLSTMCell`), we avoid if-else statements in the forward pass.

The `TreeLSTM` class (among other things) pre-calculates an initial $h$ and $c$ for every word in the sentence. Since the initial left and right child are 0, the only calculations we need to do are based on $x$, and we can drop the forget gate calculation (`prev_c_l` and `prev_c_r` are zero). The calculations we do in order to initalize $h$ and $c$ are then:

$$
c_1 =  W^{(u)}x_1 \\
o_1 = \sigma (W^{(i)}x_1) \\
h_1 = o_1 \odot \text{tanh}(c_1)$$
*NB: note that these equations are chosen as initializations of $c$ and $h$, other initializations are possible and might work equally well.*

**Sentence Representations**

All our leaf nodes are now initialized, so we can start processing the sentence in its tree form. Each sentence is represented by a buffer (initially a list with a concatenation of $[h_1, c_1]$ for every word in the reversed sentence), a stack (initially an empty list) and a transition sequence. To encode our sentence, we construct the tree from its transition sequence as explained earlier. 

*A short example that constructs a tree:*

We loop over the time dimension of the batched transition sequences (i.e. row by row), which contain values of 0's, 1's and 2's (representing SHIFT, REDUCE and padding respectively). If we have a batch of size 2 where the first example has a transition sequence given by [0, 0, 1, 0, 0, 0, 1] and the second by [0, 0, 1, 0, 0, 1], our transition batch will be given by the following two-dimensional numpy array:

$$
\text{transitions} = 
\begin{pmatrix}
0 & 0\\ 
0 & 0\\ 
1 & 1\\ 
0 & 0\\ 
0 & 0\\ 
0 & 1\\ 
1 & 2
\end{pmatrix}
$$
The inner loop (`for transition, buffer, stack in zip(t_batch, buffers, stacks)`) goes over each example in the batch and updates its buffer and stack. The nested loop for this example will then do roughy the following:

```
Time = 0:  t_batch = [0, 0], the inner loop performs 2 SHIFTs. 

Time = 1:  t_batch = [0, 0], ".."

Time = 2:  t_batch = [1, 1], causing the inner loop to fill the list child_l and child_r for both examples in the batch. Now the statement if child_l will return True, triggering a REDUCE action to be performed by our Tree LSTM cell with a batch size of 2. 

Time = 3:  t_batch = [0, 0], "..".

Time = 4:  t_batch = [0, 0], ".."

Time = 5:  t_batch = [0, 1], one SHIFT will be done and another REDUCE action will be performed by our Tree LSTM, this time of batch size 1.  

Time = 6:  t_batch = [1, 2], triggering another REDUCE action with batch size 1.
```
*NB: note that this was an artificial example for the purpose of demonstrating parts of the code, the transition sequences do not necessarily represent actual trees.*

**Batching and Unbatching**

Within the body of the outer loop over time, we use the functions for batching and unbatching. 

*Batching*

Before passing two lists of children to the reduce layer (an instance of `TreeLSTMCell`), we batch the children as they are at this point a list of tensors of variable length based on how many REDUCE actions there are to perform at a certain time step across the batch (let's call the length `L`). To do an efficient forward pass we want to transform the list to a pair of tensors of shape `([L, D], [L, D])`, which the function `batch` achieves. 

*Unbatching*

In the same line where we batched the children, we unbatch the output of the forward pass to become a list of states of length `L` again. We do this because we need to loop over each example's transition at the current time step and push the children that are reduced into a parent to the stack.

*The batch and unbatch functions let us switch between the "PyTorch world" (Tensors) and the Python world (easy to manipulate lists).*


In [106]:
# Helper functions for batching and unbatching states
# For speed we want to combine computations by batching, but 
# for processing logic we want to turn the output into lists again
# to easily manipulate.

def batch(states):
  """
  Turns a list of states into a single tensor for fast processing. 
  This function also chunks (splits) each state into a (h, c) pair"""
  return torch.cat(states, 0).chunk(2, 1)

def unbatch(state):
  """
  Turns a tensor back into a list of states.
  First, (h, c) are merged into a single state.
  Then the result is split into a list of sentences.
  """
  return torch.split(torch.cat(state, 1), 1, 0)

Take some time to understand the class below, having read the explanation above.

In [107]:
class TreeLSTM(nn.Module):
  """Encodes a sentence using a TreeLSTMCell"""

  def __init__(self, input_size, hidden_size, bias=True):
    """Creates the weights for this LSTM"""
    super(TreeLSTM, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    self.reduce = TreeLSTMCell(input_size, hidden_size)

    # project word to initial c
    self.proj_x = nn.Linear(input_size, hidden_size)
    self.proj_x_gate = nn.Linear(input_size, hidden_size)
    
    self.buffers_dropout = nn.Dropout(p=0.5)

  def forward(self, x, transitions):
    """
    WARNING: assuming x is reversed!
    :param x: word embeddings [B, T, E]
    :param transitions: [2T-1, B]
    :return: root states
    """

    B = x.size(0)  # batch size
    T = x.size(1)  # time

    # compute an initial c and h for each word
    # Note: this corresponds to input x in the Tai et al. Tree LSTM paper.
    # We do not handle input x in the TreeLSTMCell itself.
    buffers_c = self.proj_x(x)
    buffers_h = buffers_c.tanh()
    buffers_h_gate = self.proj_x_gate(x).sigmoid()
    buffers_h = buffers_h_gate * buffers_h
    
    # concatenate h and c for each word
    buffers = torch.cat([buffers_h, buffers_c], dim=-1)

    D = buffers.size(-1) // 2

    # we turn buffers into a list of stacks (1 stack for each sentence)
    # first we split buffers so that it is a list of sentences (length B)
    # then we split each sentence to be a list of word vectors
    buffers = buffers.split(1, dim=0)  # Bx[T, 2D]
    buffers = [list(b.squeeze(0).split(1, dim=0)) for b in buffers]  # BxTx[2D]

    # create B empty stacks
    stacks = [[] for _ in buffers]

    # t_batch holds 1 transition for each sentence
    for t_batch in transitions:

      child_l = []  # contains the left child for each sentence with reduce action
      child_r = []  # contains the corresponding right child

      # iterate over sentences in the batch
      # each has a transition t, a buffer and a stack
      for transition, buffer, stack in zip(t_batch, buffers, stacks):
        if transition == SHIFT:
          stack.append(buffer.pop())
        elif transition == REDUCE:
          assert len(stack) >= 2, \
            "Stack too small! Should not happen with valid transition sequences"
          child_r.append(stack.pop())  # right child is on top
          child_l.append(stack.pop())

      # if there are sentences with reduce transition, perform them batched
      if child_l:
        reduced = iter(unbatch(self.reduce(batch(child_l), batch(child_r))))
        for transition, stack in zip(t_batch, stacks):
          if transition == REDUCE:
            stack.append(next(reduced))

    final = [stack.pop().chunk(2, -1)[0] for stack in stacks]
    final = torch.cat(final, dim=0)  # tensor [B, D]

    return final

Just like the LSTM before, we will need an extra class that does the classifications.

In [108]:
class TreeLSTMClassifier(nn.Module):
  """Encodes sentence with a TreeLSTM and projects final hidden state"""

  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, vocab):
    super(TreeLSTMClassifier, self).__init__()
    self.vocab = vocab
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
    self.treelstm = TreeLSTM(embedding_dim, hidden_dim)
    self.output_layer = nn.Sequential(     
        nn.Dropout(p=0.5),
        nn.Linear(hidden_dim, output_dim, bias=True)
    )

  def forward(self, x):
    
    # x is a pair here of words and transitions; we unpack it here.
    # x is batch-major: [B, T], transitions is time major [2T-1, B]
    x, transitions = x
    emb = self.embed(x)
    
    # we use the root/top state of the Tree LSTM to classify the sentence
    root_states = self.treelstm(emb, transitions)

    # we use the last hidden state to classify the sentence
    logits = self.output_layer(root_states)
    return logits

## Special `prepare` function for Tree-LSTM

We need yet another `prepare` function. For our implementation, sentences need to be *reversed*. We will do that here.

In [109]:
def prepare_treelstm_minibatch(mb, vocab, Shuffle = False):
  """
  Returns sentences reversed (last word first)
  Returns transitions together with the sentences.  
  """
  batch_size = len(mb)
  maxlen = max([len(ex.tokens) for ex in mb])
    
  # vocab returns 0 if the word is not there
  # NOTE: reversed sequence!
  x = [pad([vocab.w2i.get(t, 0) for t in ex.tokens], maxlen)[::-1] for ex in mb]
  
  x = torch.LongTensor(x)
  x = x.to(device)
  
  y = [ex.label for ex in mb]
  y = torch.LongTensor(y)
  y = y.to(device)
  
  maxlen_t = max([len(ex.transitions) for ex in mb])
  transitions = [pad(ex.transitions, maxlen_t, pad_value=2) for ex in mb]
  transitions = np.array(transitions)
  transitions = transitions.T  # time-major
  
  return (x, transitions), y

## Training

In [110]:
# Now let's train the Tree LSTM!

tree_model = TreeLSTMClassifier(
    len(v.w2i), 300, 150, len(t2i), v)

with torch.no_grad():
  tree_model.embed.weight.data.copy_(torch.from_numpy(vectors))
  tree_model.embed.weight.requires_grad = False
  
def do_train(model):
  
  print(model)
  print_parameters(model)

  model = model.to(device)

  optimizer = optim.Adam(model.parameters(), lr=2e-4)
  
  return train_model(
      model, optimizer, num_iterations=30000, 
      print_every=250, eval_every=250,
      prep_fn=prepare_treelstm_minibatch,
      eval_fn=evaluate,
      batch_fn=get_minibatch,
      batch_size=25, eval_batch_size=25)
  
results = do_train(tree_model)

TreeLSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (treelstm): TreeLSTM(
    (reduce): TreeLSTMCell(300, 150)
    (proj_x): Linear(in_features=300, out_features=150, bias=True)
    (proj_x_gate): Linear(in_features=300, out_features=150, bias=True)
    (buffers_dropout): Dropout(p=0.5, inplace=False)
  )
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=150, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
treelstm.reduce.reduce_layer.weight [750, 300]   requires_grad=True
treelstm.reduce.reduce_layer.bias [750]        requires_grad=True
treelstm.proj_x.weight   [150, 300]   requires_grad=True
treelstm.proj_x.bias     [150]        requires_grad=True
treelstm.proj_x_gate.weight [150, 300]   requires_grad=True
treelstm.proj_x_gate.bias [150]        requires_grad=True
output_layer.1.weight    [5, 150]     requires_grad=True
output_layer.1.bias      [5]          requires_grad=True

T

KeyboardInterrupt: 

In [ ]:
# plot

# Further experiments and report

For your report, you are expected to answer research questions by doing further experiments.

## Research Questions

Make sure you cover at least the following:

- How important is word order for this task?
- Does the tree structure help to get a better accuracy?
- How does performance depend on the sentence length? Compare the various models. Is there a model that does better on longer sentences? If so, why?
- Do you get better performance if you supervise the sentiment **at each node in the tree**? You can extract more training examples by treating every node in each tree as a separate tree. You will need to write a function that extracts all subtrees given a treestring. 
    - Warning: NLTK's Tree function seems to result in invalid trees in some cases, so be careful if you want to parse the string to a tree structure before extraction the phrases.

**To obtain a full grade, you should conduct further investigations.** For example, you can also investigate the following:

- When making a wrong prediction, can you figure out at what point in the tree (sentence) the model fails? You can make a prediction at each node to investigate.
- How does N-ary Tree LSTM compare to the Child-Sum Tree LSTM? 
- How do the Tai et al. Tree LSTMs compare to Le & Zuidema's formulation?
- Or... your own research question!

In general:

- ***When you report numbers, please report the mean accuracy across 3 (or more) runs with different random seed, together with the standard deviation.*** This is because the final performance may vary per random seed. 
More precisely, you should run each model with 3 different seeds, and for each of these 3 runs, evaluate the best model (according to the validation) on the test dataset. The validation dataset is used for finding the best model over iterations, but the accuracy you report should be on the test dataset.

## Report instructions

Your report needs to be written in LaTeX. You are required to use the ACL 2020 template which you can download from or edit directly on [Overleaf](https://www.overleaf.com/latex/templates/instructions-for-acl-2018-proceedings/xzmhqgnmkppc). Make sure your names and student numbers are visible at the top. (Tip: you need to uncomment `\aclfinalcopy`).
You can find some general tips about writing a research paper [here](https://www.microsoft.com/en-us/research/academic-program/write-great-research-paper/), but note that you need to make your own judgment about what is appropriate for this project. 

We expect you to use the following structure:
1. Introduction (~1 page) - describe the problem, your research questions and goals, a summary of your findings and contributions. Please cite related work (models, data set) as part of your introduction here, since this is a short paper.
    - Introduce the task and the main goal
    - Clear research questions
    - Motivating the importance of the questions and explaining the expectations
    - How are these addressed or not addressed in the literature
    - What is your approach
    - Short summary of your findings
2. Background (~1/2-1 page) -
cover the main techniques ("building blocks") used in your project (e.g. word embeddings, LSTM, Tree-LSTM) and intuitions behind them. Be accurate and concise.
    - How each technique that you use works (don't just copy the formulas)
    - The relation between the techniques
3. Models (~1/2 page) - Cover the models that you used.
    - The architecture of the final models (How do you use LSTM or Tree-LSTM for the sentiment classification task? What layers do you have, how do you do classification? What is your loss function?)
4. Experiments (~1/2 page) - Describe your experimental setup. The information here should allow someone else to reproduce your experiments. Describe how you evaluate the models.
    - Explain the task and the data
    - Training the models (model, data, parameters and hyper parameters if the models, training algorithms, what supervision signals you use, etc.)
    - Evaluation (e.g. metrics)
5. Results and Analysis (~1 page). Go over the results and analyse your findings.
    - Answer each of the research questions you raised in the introduction.
    - Plots and figures highlighting interesting patterns
    - What are the factors that make model A better than model B in task C? Investigate to prove their effect!
6. Conclusion (~1/4 page). The main conclusions of your experiments.
    - What have you learned from you experiments? How does it relate to what is already known in the literature?
    - Were the results as expected? Any surprising results? Why?
    - Based on what you learned what would you suggest doing next?


General Tips:

- Math notation – define each variable (either in running text, or in a pseudo-legenda after or before the equation).
- Define technical terminology you need.
- Avoid colloquial language – everything can be said in a scientific-sounding way.
- Avoid lengthy sentences, stay to the point.
- Do not spend space on "obvious" things.
- Do not go over the page limit. (We will deduct points for that.)
- The page limit is 4 pages excluding references and appendix. There is no strict limit to references and appendix. However, the report needs to remain fully self-contained: the appendix should only include content that is not necessary to understand your work. For example, preprocessing decisions, model parameters, pseudocode, sample system inputs/outputs, and other details that are necessary for the exact replication of your work can be put into the appendix. However, 


An ideal report:
- Precise, scientific-sounding, technical, to the point 
  - Little general “waffle”/chit-chat
- Not boring – because you don’t explain obvious things too much
- Efficient delivery of (only) the facts that we need to know to understand/reimplement
- Results visually well-presented and described with the correct priority of importance of sub-results
- Insightful analysis – speculation should connect to something interesting and not be too much; the reader “learns something new”
- No typos, no colloquialisms – well-considered language
- This normally means several re-draftings (re-orderings of information)


## How does normal LSTM handle word-order?

In [111]:
### Normal LSTM RUN
def train_LSTM_Noshuffle():
    lstm_model = LSTMClassifier(len(v.w2i), 300, 168, len(t2i), v)

    with torch.no_grad():
        lstm_model.embed.weight.data.copy_(torch.from_numpy(vectors))
        lstm_model.embed.weight.requires_grad = False

    print(lstm_model)
    print_parameters(lstm_model)

    lstm_model = lstm_model.to(device)
    optimizer = optim.Adam(lstm_model.parameters(), lr=3e-4)

    acc = train_model(
        lstm_model, optimizer, num_iterations=25000, 
        print_every=250, eval_every=1000, OnlyTest=True)
    return  acc

In [112]:
### Shuffled Word-order
def train_LSTM_Shuffle():
    lstm_model = LSTMClassifier(len(v.w2i), 300, 168, len(t2i), v)

    with torch.no_grad():
        lstm_model.embed.weight.data.copy_(torch.from_numpy(vectors))
        lstm_model.embed.weight.requires_grad = False

    print(lstm_model)
    print_parameters(lstm_model)

    lstm_model = lstm_model.to(device)
    optimizer = optim.Adam(lstm_model.parameters(), lr=3e-4)

    acc = train_model(
        lstm_model, optimizer, num_iterations=25000, 
        print_every=250, eval_every=1000, Shuffle = True, OnlyTest = True)
    return acc

In [196]:
Seeds = [8,88,888]
NoShuffleAcc= []
for seed in Seeds:
    np.random.seed(seed)
    torch.manual_seed(seed)
    NoShuffleAcc.append(train_LSTM_Noshuffle())
   


LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.W_ii                 [168, 300]   requires_grad=True
rnn.W_hi                 [168, 168]   requires_grad=True
rnn.B_ii                 [168]        requires_grad=True
rnn.B_hi                 [168]        requires_grad=True
rnn.W_if                 [168, 300]   requires_grad=True
rnn.W_hf                 [168, 168]   requires_grad=True
rnn.B_if                 [168]        requires_grad=True
rnn.B_hf                 [168]        requires_grad=True
rnn.W_ig                 [168, 300]   requires_grad=True
rnn.W_hg                 [168, 168]   requires_grad=True
rnn.B_ig                 [168]        requires_grad=True
rnn.B_hg                 [168]        requires_grad=True
rnn.W_io                 [168

Iter 3500: loss=334.8044, time=198.69s
Iter 3750: loss=329.6274, time=208.71s
Iter 4000: loss=343.7386, time=219.20s
iter 4000: dev acc=0.4114
new highscore
Iter 4250: loss=336.2119, time=246.37s
Iter 4500: loss=340.0977, time=257.31s
Iter 4750: loss=334.1170, time=267.87s
Iter 5000: loss=357.7605, time=278.22s
iter 5000: dev acc=0.3951
Iter 5250: loss=337.4934, time=304.93s
Iter 5500: loss=325.9464, time=315.50s
Iter 5750: loss=324.0919, time=326.26s
Iter 6000: loss=337.8101, time=336.49s
iter 6000: dev acc=0.4024
Iter 6250: loss=352.8556, time=363.01s
Iter 6500: loss=337.4892, time=373.50s
Iter 6750: loss=322.9730, time=384.15s
Iter 7000: loss=331.8418, time=393.25s
iter 7000: dev acc=0.4087
Iter 7250: loss=338.4322, time=411.00s
Iter 7500: loss=323.0828, time=418.15s
Iter 7750: loss=342.2150, time=425.39s
Iter 8000: loss=331.4451, time=432.58s
iter 8000: dev acc=0.4015
Iter 8250: loss=329.6336, time=449.23s
Iter 8500: loss=332.0044, time=456.32s
Shuffling training data
Iter 8750: lo

Iter 14000: loss=322.1998, time=505.19s
iter 14000: dev acc=0.4450
new highscore
Iter 14250: loss=307.0634, time=521.15s
Iter 14500: loss=305.0211, time=528.08s
Iter 14750: loss=321.0978, time=535.28s
Iter 15000: loss=322.2107, time=542.01s
iter 15000: dev acc=0.4496
new highscore
Iter 15250: loss=316.4579, time=558.17s
Iter 15500: loss=327.3564, time=565.06s
Iter 15750: loss=316.8428, time=571.96s
Iter 16000: loss=305.0984, time=578.75s
iter 16000: dev acc=0.4514
new highscore
Iter 16250: loss=321.0566, time=595.00s
Iter 16500: loss=307.1908, time=602.20s
Iter 16750: loss=314.3990, time=609.17s
Iter 17000: loss=312.0493, time=615.73s
iter 17000: dev acc=0.4096
Shuffling training data
Iter 17250: loss=309.8795, time=632.12s
Iter 17500: loss=283.7558, time=639.01s
Iter 17750: loss=305.5788, time=646.17s
Iter 18000: loss=307.2886, time=652.89s
iter 18000: dev acc=0.4387
Iter 18250: loss=296.0871, time=669.03s
Iter 18500: loss=287.7024, time=676.38s
Iter 18750: loss=303.4905, time=683.52s

0.46108597285067876 0.0
0.46108597285067876 0.0


In [197]:
Seeds = [8,88,888]
ShuffleAcc= []
for seed in Seeds:
    np.random.seed(seed)
    torch.manual_seed(seed)
    ShuffleAcc.append(train_LSTM_Shuffle())


LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.W_ii                 [168, 300]   requires_grad=True
rnn.W_hi                 [168, 168]   requires_grad=True
rnn.B_ii                 [168]        requires_grad=True
rnn.B_hi                 [168]        requires_grad=True
rnn.W_if                 [168, 300]   requires_grad=True
rnn.W_hf                 [168, 168]   requires_grad=True
rnn.B_if                 [168]        requires_grad=True
rnn.B_hf                 [168]        requires_grad=True
rnn.W_ig                 [168, 300]   requires_grad=True
rnn.W_hg                 [168, 168]   requires_grad=True
rnn.B_ig                 [168]        requires_grad=True
rnn.B_hg                 [168]        requires_grad=True
rnn.W_io                 [168

Iter 4000: loss=331.0206, time=137.31s
iter 4000: dev acc=0.3887
new highscore
Iter 4250: loss=345.2162, time=153.53s
Iter 4500: loss=341.9814, time=160.36s
Iter 4750: loss=331.9848, time=167.12s
Iter 5000: loss=340.5694, time=173.88s
iter 5000: dev acc=0.3924
new highscore
Iter 5250: loss=333.6733, time=190.23s
Iter 5500: loss=343.3989, time=197.03s
Iter 5750: loss=328.8161, time=203.52s
Iter 6000: loss=331.7087, time=210.07s
iter 6000: dev acc=0.3969
new highscore
Iter 6250: loss=340.0904, time=226.30s
Iter 6500: loss=331.8474, time=233.62s
Iter 6750: loss=325.2798, time=240.61s
Iter 7000: loss=347.0141, time=247.21s
iter 7000: dev acc=0.4087
new highscore
Iter 7250: loss=326.2858, time=263.29s
Iter 7500: loss=309.3582, time=270.28s
Iter 7750: loss=327.5449, time=277.07s
Iter 8000: loss=316.9572, time=283.63s
iter 8000: dev acc=0.4178
new highscore
Iter 8250: loss=336.7771, time=299.79s
Iter 8500: loss=335.8022, time=306.61s
Shuffling training data
Iter 8750: loss=323.8654, time=313.

Iter 14000: loss=329.1345, time=506.51s
iter 14000: dev acc=0.4233
Iter 14250: loss=323.7808, time=522.26s
Iter 14500: loss=309.4560, time=529.16s
Iter 14750: loss=328.3927, time=536.09s
Iter 15000: loss=300.8195, time=543.04s
iter 15000: dev acc=0.4332
Iter 15250: loss=304.4755, time=558.96s
Iter 15500: loss=324.1941, time=566.10s
Iter 15750: loss=313.7164, time=573.15s
Iter 16000: loss=325.0499, time=579.92s
iter 16000: dev acc=0.4332
Iter 16250: loss=331.6606, time=596.98s
Iter 16500: loss=313.8522, time=604.63s
Iter 16750: loss=327.5697, time=611.80s
Iter 17000: loss=319.7533, time=619.70s
iter 17000: dev acc=0.4332
Shuffling training data
Iter 17250: loss=303.8649, time=636.87s
Iter 17500: loss=297.5353, time=643.77s
Iter 17750: loss=304.5427, time=650.66s
Iter 18000: loss=307.3457, time=657.52s
iter 18000: dev acc=0.4487
new highscore
Iter 18250: loss=327.3720, time=673.40s
Iter 18500: loss=308.4418, time=680.66s
Iter 18750: loss=304.8086, time=688.14s
Iter 19000: loss=311.7419, 

In [198]:
print(np.average(NoShuffleAcc), np.std(NoShuffleAcc))
print(np.average(ShuffleAcc), np.std(ShuffleAcc))

print(NoShuffleAcc)
print(ShuffleAcc)

0.4559577677224736 0.00726178166950872
0.45656108597285067 0.005172376530763873
[0.4606334841628959, 0.46153846153846156, 0.4457013574660634]
[0.4493212669683258, 0.4592760180995475, 0.46108597285067876]


### EXPERIMENT LSTM VS TREE LSTM

In [207]:
# TREE LSTM MINIBATCH
# Now let's train the Tree LSTM!
def tree_lstm(data = train_data):
    tree_model = TreeLSTMClassifier(
        len(v.w2i), 300, 150, len(t2i), v)

    with torch.no_grad():
      tree_model.embed.weight.data.copy_(torch.from_numpy(vectors))
      tree_model.embed.weight.requires_grad = False

    def do_train(model):

      print(model)
      print_parameters(model)

      model = model.to(device)

      optimizer = optim.Adam(model.parameters(), lr=2e-4)

      return train_model(
          model, optimizer, num_iterations=30000, 
          print_every=250, eval_every=250,
          prep_fn=prepare_treelstm_minibatch,
          eval_fn=evaluate,
          batch_fn=get_minibatch,
          batch_size=25, eval_batch_size=25,
          OnlyTest=True,
          data = data)

    return do_train(tree_model)

In [208]:
# LSTM MINIBATCH
def lstm_minibatch(data = train_data):
    lstm_model = LSTMClassifier(
        len(v.w2i), 300, 168, len(t2i), v)

    # copy pre-trained vectors into embeddings table
    with torch.no_grad():
      lstm_model.embed.weight.data.copy_(torch.from_numpy(vectors))
      lstm_model.embed.weight.requires_grad = False

    print(lstm_model)
    print_parameters(lstm_model)  

    lstm_model = lstm_model.to(device)

    batch_size = 25
    optimizer = optim.Adam(lstm_model.parameters(), lr=2e-4)

    lstm_accuracies = train_model(
        lstm_model, optimizer, num_iterations=30000, 
        print_every=250, eval_every=250,
        batch_size=batch_size,
        batch_fn=get_minibatch, 
        prep_fn=prepare_minibatch,
        eval_fn=evaluate,
        OnlyTest=True, 
        data = data)
    
    return lstm_accuracies

In [209]:
Seeds = [8,88,888]
minibatch_lstm = []
minibatch_tree_lstm = []

for seed in Seeds:
    np.random.seed(seed)
    torch.manual_seed(seed)
    minibatch_lstm.append(lstm_minibatch())
    minibatch_tree_lstm.append(tree_lstm())
   


LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.W_ii                 [168, 300]   requires_grad=True
rnn.W_hi                 [168, 168]   requires_grad=True
rnn.B_ii                 [168]        requires_grad=True
rnn.B_hi                 [168]        requires_grad=True
rnn.W_if                 [168, 300]   requires_grad=True
rnn.W_hf                 [168, 168]   requires_grad=True
rnn.B_if                 [168]        requires_grad=True
rnn.B_hf                 [168]        requires_grad=True
rnn.W_ig                 [168, 300]   requires_grad=True
rnn.W_hg                 [168, 168]   requires_grad=True
rnn.B_ig                 [168]        requires_grad=True
rnn.B_hg                 [168]        requires_grad=True
rnn.W_io                 [168

Iter 20500: loss=22.3700, time=1099.94s
iter 20500: dev acc=0.4087
Shuffling training data
Iter 20750: loss=21.6921, time=1113.31s
iter 20750: dev acc=0.4105
Shuffling training data
Iter 21000: loss=20.4541, time=1126.57s
iter 21000: dev acc=0.3924
Shuffling training data
Iter 21250: loss=22.8347, time=1139.81s
iter 21250: dev acc=0.4078
Iter 21500: loss=25.0489, time=1153.07s
iter 21500: dev acc=0.4033
Shuffling training data
Iter 21750: loss=17.5420, time=1166.30s
iter 21750: dev acc=0.3987
Shuffling training data
Iter 22000: loss=18.5435, time=1179.52s
iter 22000: dev acc=0.4251
Shuffling training data
Iter 22250: loss=18.1910, time=1192.77s
iter 22250: dev acc=0.4169
Iter 22500: loss=12.8475, time=1206.08s
iter 22500: dev acc=0.4114
Shuffling training data
Iter 22750: loss=31.9940, time=1219.48s
iter 22750: dev acc=0.3978
Shuffling training data
Iter 23000: loss=19.6550, time=1232.78s
iter 23000: dev acc=0.3978
Iter 23250: loss=12.5918, time=1245.99s
iter 23250: dev acc=0.4060
Shuf

Shuffling training data
Iter 11500: loss=104.9235, time=801.99s
iter 11500: dev acc=0.4005
Shuffling training data
Iter 11750: loss=104.2607, time=819.49s
iter 11750: dev acc=0.4187
Shuffling training data
Iter 12000: loss=99.2736, time=836.94s
iter 12000: dev acc=0.4069
Iter 12250: loss=91.3534, time=854.33s
iter 12250: dev acc=0.3987
Shuffling training data
Iter 12500: loss=85.1089, time=871.64s
iter 12500: dev acc=0.4078
Shuffling training data
Iter 12750: loss=82.2052, time=889.32s
iter 12750: dev acc=0.3896
Shuffling training data
Iter 13000: loss=79.5677, time=906.72s
iter 13000: dev acc=0.4069
Iter 13250: loss=73.3628, time=924.08s
iter 13250: dev acc=0.3869
Shuffling training data
Iter 13500: loss=73.9643, time=941.65s
iter 13500: dev acc=0.3960
Shuffling training data
Iter 13750: loss=74.7704, time=959.13s
iter 13750: dev acc=0.4178
Iter 14000: loss=74.3890, time=976.59s
iter 14000: dev acc=0.3915
Shuffling training data
Iter 14250: loss=60.9535, time=993.80s
iter 14250: dev a

Shuffling training data
Iter 1500: loss=306.6423, time=86.14s
iter 1500: dev acc=0.4432
new highscore
Shuffling training data
Iter 1750: loss=305.6896, time=99.90s
iter 1750: dev acc=0.4214
Iter 2000: loss=300.6141, time=113.41s
iter 2000: dev acc=0.4505
new highscore
Shuffling training data
Iter 2250: loss=295.8692, time=127.65s
iter 2250: dev acc=0.4505
Shuffling training data
Iter 2500: loss=290.1920, time=141.54s
iter 2500: dev acc=0.4532
new highscore
Shuffling training data
Iter 2750: loss=289.9603, time=155.37s
iter 2750: dev acc=0.4450
Iter 3000: loss=280.7353, time=169.01s
iter 3000: dev acc=0.4578
new highscore
Shuffling training data
Iter 3250: loss=282.5368, time=182.99s
iter 3250: dev acc=0.4332
Shuffling training data
Iter 3500: loss=275.3035, time=197.24s
iter 3500: dev acc=0.4668
new highscore
Iter 3750: loss=272.9972, time=211.88s
iter 3750: dev acc=0.4550
Shuffling training data
Iter 4000: loss=263.8719, time=226.98s
iter 4000: dev acc=0.4532
Shuffling training data
I

Iter 25750: loss=17.8931, time=1451.27s
iter 25750: dev acc=0.3942
Shuffling training data
Iter 26000: loss=11.0306, time=1464.98s
iter 26000: dev acc=0.3951
Iter 26250: loss=8.9817, time=1478.71s
iter 26250: dev acc=0.3960
Shuffling training data
Iter 26500: loss=10.7764, time=1492.32s
iter 26500: dev acc=0.4105
Shuffling training data
Iter 26750: loss=12.8996, time=1506.27s
iter 26750: dev acc=0.3942
Iter 27000: loss=24.8075, time=1519.95s
iter 27000: dev acc=0.4124
Shuffling training data
Iter 27250: loss=13.7849, time=1533.68s
iter 27250: dev acc=0.4142
Shuffling training data
Iter 27500: loss=15.2165, time=1547.38s
iter 27500: dev acc=0.4033
Shuffling training data
Iter 27750: loss=13.3261, time=1561.19s
iter 27750: dev acc=0.3987
Iter 28000: loss=6.8011, time=1574.87s
iter 28000: dev acc=0.4087
Shuffling training data
Iter 28250: loss=6.1872, time=1588.79s
iter 28250: dev acc=0.4060
Shuffling training data
Iter 28500: loss=6.6577, time=1602.33s
iter 28500: dev acc=0.4015
Shufflin

Iter 16750: loss=48.7183, time=1285.02s
iter 16750: dev acc=0.4196
Shuffling training data
Iter 17000: loss=35.8695, time=1304.66s
iter 17000: dev acc=0.4087
Shuffling training data
Iter 17250: loss=34.2138, time=1324.18s
iter 17250: dev acc=0.4105
Shuffling training data
Iter 17500: loss=38.3143, time=1343.80s
iter 17500: dev acc=0.4124
Iter 17750: loss=33.0136, time=1363.39s
iter 17750: dev acc=0.4305
Shuffling training data
Iter 18000: loss=28.2504, time=1383.35s
iter 18000: dev acc=0.4133
Shuffling training data
Iter 18250: loss=26.1698, time=1402.87s
iter 18250: dev acc=0.4078
Shuffling training data
Iter 18500: loss=30.6340, time=1422.45s
iter 18500: dev acc=0.4223
Iter 18750: loss=25.6644, time=1442.17s
iter 18750: dev acc=0.4169
Shuffling training data
Iter 19000: loss=29.2300, time=1461.35s
iter 19000: dev acc=0.4060
Shuffling training data
Iter 19250: loss=32.3723, time=1481.26s
iter 19250: dev acc=0.3969
Shuffling training data
Iter 19500: loss=25.1579, time=1501.23s
iter 19

iter 6500: dev acc=0.4287
Iter 6750: loss=193.8960, time=420.11s
iter 6750: dev acc=0.4460
Shuffling training data
Iter 7000: loss=185.6835, time=434.62s
iter 7000: dev acc=0.4414
Shuffling training data
Iter 7250: loss=183.9314, time=449.95s
iter 7250: dev acc=0.4441
Iter 7500: loss=175.6723, time=466.77s
iter 7500: dev acc=0.4332
Shuffling training data
Iter 7750: loss=169.4854, time=481.98s
iter 7750: dev acc=0.4396
Shuffling training data
Iter 8000: loss=157.2652, time=498.13s
iter 8000: dev acc=0.4360
Shuffling training data
Iter 8250: loss=154.8145, time=513.61s
iter 8250: dev acc=0.4360
Iter 8500: loss=148.4717, time=528.60s
iter 8500: dev acc=0.4269
Shuffling training data
Iter 8750: loss=132.2384, time=544.62s
iter 8750: dev acc=0.4269
Shuffling training data
Iter 9000: loss=134.8415, time=560.86s
iter 9000: dev acc=0.4260
Shuffling training data
Iter 9250: loss=124.9475, time=576.32s
iter 9250: dev acc=0.4278
Iter 9500: loss=116.7158, time=591.71s
iter 9500: dev acc=0.4351
Sh

Iter 250: loss=385.9315, time=18.58s
iter 250: dev acc=0.3787
new highscore
Shuffling training data
Iter 500: loss=336.1137, time=38.82s
iter 500: dev acc=0.4133
new highscore
Shuffling training data
Iter 750: loss=321.6774, time=59.03s
iter 750: dev acc=0.4305
new highscore
Iter 1000: loss=312.4828, time=79.79s
iter 1000: dev acc=0.4351
new highscore
Shuffling training data
Iter 1250: loss=308.8541, time=100.01s
iter 1250: dev acc=0.4414
new highscore
Shuffling training data
Iter 1500: loss=302.7437, time=120.83s
iter 1500: dev acc=0.4596
new highscore
Shuffling training data
Iter 1750: loss=302.1606, time=141.42s
iter 1750: dev acc=0.4278
Iter 2000: loss=296.9306, time=161.35s
iter 2000: dev acc=0.4605
new highscore
Shuffling training data
Iter 2250: loss=292.6952, time=182.08s
iter 2250: dev acc=0.4569
Shuffling training data
Iter 2500: loss=287.9734, time=204.91s
iter 2500: dev acc=0.4378
Shuffling training data
Iter 2750: loss=287.8550, time=226.47s
iter 2750: dev acc=0.4487
Iter 

Shuffling training data
Iter 24500: loss=9.6884, time=1824.31s
iter 24500: dev acc=0.3942
Shuffling training data
Iter 24750: loss=8.9342, time=1841.58s
iter 24750: dev acc=0.3996
Shuffling training data
Iter 25000: loss=10.8503, time=1859.03s
iter 25000: dev acc=0.4042
Iter 25250: loss=31.9063, time=1877.34s
iter 25250: dev acc=0.3851
Shuffling training data
Iter 25500: loss=22.9908, time=1895.07s
iter 25500: dev acc=0.3869
Shuffling training data
Iter 25750: loss=13.0824, time=1915.89s
iter 25750: dev acc=0.3960
Shuffling training data
Iter 26000: loss=8.0668, time=1934.29s
iter 26000: dev acc=0.3942
Iter 26250: loss=7.1837, time=1954.20s
iter 26250: dev acc=0.3933
Shuffling training data
Iter 26500: loss=7.7141, time=1972.67s
iter 26500: dev acc=0.3996
Shuffling training data
Iter 26750: loss=8.4155, time=1990.96s
iter 26750: dev acc=0.4033
Iter 27000: loss=18.6598, time=2009.38s
iter 27000: dev acc=0.4114
Shuffling training data
Iter 27250: loss=17.0359, time=2028.47s
iter 27250: d

In [213]:
print(minibatch_lstm, minibatch_tree_lstm)

[0.46968325791855203, 0.4751131221719457, 0.47420814479638007] [0.481447963800905, 0.47149321266968325, 0.47104072398190044]


### Sentence length experiment

In [210]:
Seeds = [1, 29, 88]

minibatch_lstm_short = []
minibatch_lstm_long = []

minibatch_tree_lstm_short = []
minibatch_tree_lstm_long = []

for seed in Seeds:
    np.random.seed(seed)
    torch.manual_seed(seed)
    minibatch_lstm_short.append(lstm_minibatch(short_train))
    minibatch_lstm_long.append(lstm_minibatch(long_train))
    
    minibatch_tree_lstm_short.append(tree_lstm(short_train))
    minibatch_tree_lstm_long.append(tree_lstm(long_train))
    
    


LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.W_ii                 [168, 300]   requires_grad=True
rnn.W_hi                 [168, 168]   requires_grad=True
rnn.B_ii                 [168]        requires_grad=True
rnn.B_hi                 [168]        requires_grad=True
rnn.W_if                 [168, 300]   requires_grad=True
rnn.W_hf                 [168, 168]   requires_grad=True
rnn.B_if                 [168]        requires_grad=True
rnn.B_hf                 [168]        requires_grad=True
rnn.W_ig                 [168, 300]   requires_grad=True
rnn.W_hg                 [168, 168]   requires_grad=True
rnn.B_ig                 [168]        requires_grad=True
rnn.B_hg                 [168]        requires_grad=True
rnn.W_io                 [168

Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 10250: loss=1.4631, time=179.11s
iter 10250: dev acc=0.3406
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 10500: loss=1.1054, time=183.58s
iter 10500: dev acc=0.3406
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 10750: loss=0.8932, time=188.06s
iter 10750: dev acc=0.3379
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 11000: loss=0.7134, time=192.42s
iter 11000: dev acc=0.3370
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 11250: loss=0.6745, time=196.82s
iter 11250: dev acc=0.3361
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 11500: loss=0.5673, ti

Iter 22000: loss=0.1696, time=391.09s
iter 22000: dev acc=0.3361
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 22250: loss=0.1410, time=396.06s
iter 22250: dev acc=0.3342
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 22500: loss=0.1241, time=400.82s
iter 22500: dev acc=0.3370
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 22750: loss=0.1084, time=405.47s
iter 22750: dev acc=0.3361
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 23000: loss=0.1149, time=410.05s
iter 23000: dev acc=0.3424
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 23250: loss=0.1011, time=414.51s
iter 23250: dev acc=0.3406
Shuffling training data
Shuffling training data
Shuffling 

iter 1750: dev acc=0.3797
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 2000: loss=98.8240, time=130.99s
iter 2000: dev acc=0.3615
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 2250: loss=55.5259, time=146.60s
iter 2250: dev acc=0.3569
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 2500: loss=74.5119, time=162.30s
iter 2500: dev acc=0.3669
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 2750: loss=25.1303, time=179.81s
iter 2750: dev acc=0.3606
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 3000: loss=18.9160, time=195.35s
iter 3000: dev acc=0.3769
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 3

Shuffling training data
Shuffling training data
Iter 14000: loss=0.0844, time=889.93s
iter 14000: dev acc=0.3733
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 14250: loss=0.0785, time=906.23s
iter 14250: dev acc=0.3724
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 14500: loss=0.0588, time=922.51s
iter 14500: dev acc=0.3651
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 14750: loss=0.0498, time=938.57s
iter 14750: dev acc=0.3706
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 15000: loss=0.0457, time=954.42s
iter 15000: dev acc=0.3678
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 15250: loss=0.0447, time=970.20s
iter 15250: dev acc=0.3724
Shuffling training data
Shuffling 

...

Shuffling training data
Shuffling training data
Iter 2500: loss=44.6296, time=57.32s
iter 2500: dev acc=0.3597
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 2750: loss=30.9451, time=63.28s
iter 2750: dev acc=0.3497
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 3000: loss=22.9124, time=69.05s
iter 3000: dev acc=0.3397
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 3250: loss=25.5202, time=74.80s
iter 3250: dev acc=0.3488
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 3500: loss=31.4950, time=80.57s
iter 3500: dev acc=0.3551
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 3750: loss=12.2019, time=86.35s
iter 3750: dev acc=0.3533
Shuffling training data
Shuffling training dat

Shuffling training data
Shuffling training data
Iter 14500: loss=0.1663, time=335.28s
iter 14500: dev acc=0.3560
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 14750: loss=0.1260, time=341.03s
iter 14750: dev acc=0.3524
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 15000: loss=1.6089, time=346.81s
iter 15000: dev acc=0.3306
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 15250: loss=39.1206, time=352.59s
iter 15250: dev acc=0.3660
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 15500: loss=1.1042, time=358.35s
iter 15500: dev acc=0.3669
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 15750: loss=0.7129, time=364.14s
iter 15750: dev acc=0.3651
Shuffling

Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 26500: loss=0.1039, time=614.12s
iter 26500: dev acc=0.3688
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 26750: loss=0.0930, time=619.85s
iter 26750: dev acc=0.3678
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 27000: loss=0.0865, time=625.64s
iter 27000: dev acc=0.3669
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 27250: loss=0.0888, time=631.42s
iter 27250: dev acc=0.3669
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 27500: loss=0.0822, time=637.23s
iter 27500: dev acc=0.3688
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 27750: loss=0.0726, ti

Iter 6750: loss=0.5092, time=615.72s
iter 6750: dev acc=0.3406
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 7000: loss=0.4152, time=638.80s
iter 7000: dev acc=0.3406
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 7250: loss=0.3489, time=661.93s
iter 7250: dev acc=0.3397
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 7500: loss=0.2815, time=684.75s
iter 7500: dev acc=0.3388
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 7750: loss=0.2463, time=707.53s
iter 7750: dev acc=0.3406
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 8000: loss=0.2154, time=730.29s
iter 8000: dev acc=0.3479
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training dat

iter 18750: dev acc=0.3470
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 19000: loss=0.0999, time=1733.66s
iter 19000: dev acc=0.3497
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 19250: loss=0.0846, time=1756.50s
iter 19250: dev acc=0.3460
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 19500: loss=0.0856, time=1779.25s
iter 19500: dev acc=0.3479
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 19750: loss=0.0642, time=1802.12s
iter 19750: dev acc=0.3370
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Iter 20000: loss=0.0667, time=1825.37s
iter 20000: dev acc=0.3433
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training data
Shuffling training 

In [214]:
print(minibatch_lstm_long, minibatch_lstm_short, minibatch_tree_lstm_short, minibatch_tree_lstm_long)

[0.3941176470588235, 0.3990950226244344, 0.3945701357466063] [0.3995475113122172, 0.36787330316742084, 0.37918552036199094] [0.40316742081447965, 0.41402714932126694, 0.40407239819004526] [0.4230769230769231, 0.4158371040723982, 0.4067873303167421]
